In [1]:
# Parameters
emotion = "romantic"
story_arc = "love_story"
key_root = 62
tempo = 70
clean_mode = True
dynamic_range = "moderate"
phrase_length = "medium"
enable_layers = {
    "melody": True,
    "harmony": True,
    "bass": True,
    "strings": True,
    "sparkle": True,
}


In [2]:
pip install -q transformers accelerate torch soundfile huggingface_hub certifi

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install torch numpy pretty_midi soundfile scipy pyfluidsynth


Note: you may need to restart the kernel to use updated packages.


In [4]:
# ============================================================
# 🎹 Full Self-Contained Piano Generation Script
# ============================================================
import torch
import numpy as np
import pretty_midi
import soundfile as sf

from scipy.signal import resample

# ------------------------
# ✅ Load the trained checkpoint
# ------------------------
checkpoint_path = "hybrid_vae_transformer_model.pt"  # make sure file exists in working dir
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Using device: {device}")

checkpoint = torch.load(checkpoint_path, map_location=device)

# ------------------------
# ✅ Hybrid Model Class (must match training arch)
# ------------------------
class HybridVAETransformer(torch.nn.Module):
    def __init__(self, input_dim=4, latent_dim=128, d_model=4, nhead=4, num_layers=2):
        super().__init__()
        self.encoder = torch.nn.Sequential(
            torch.nn.Linear(input_dim, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, latent_dim * 2)
        )
        self.transformer = torch.nn.Transformer(
            d_model=d_model, nhead=nhead,
            num_encoder_layers=num_layers,
            num_decoder_layers=num_layers,
            batch_first=True
        )
        self.decoder = torch.nn.Sequential(
            torch.nn.Linear(d_model, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, input_dim)
        )

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

# ------------------------
# ✅ Restore model + config
# ------------------------
cfg = checkpoint.get("model_config", {})
input_dim = int(cfg.get("input_dim", 4))
latent_dim = int(cfg.get("latent_dim", 128))
d_model = int(cfg.get("d_model", input_dim))
seq_len = int(cfg.get("seq_len", 512))

model = HybridVAETransformer(input_dim=input_dim, latent_dim=latent_dim, d_model=d_model)
if "model_state_dict" in checkpoint:
    model.load_state_dict(checkpoint["model_state_dict"], strict=False)
    print("✅ Model weights loaded successfully.")
else:
    model.load_state_dict(checkpoint, strict=False)
    print("⚠️ Loaded raw weights (no model_state_dict key).")

model.to(device).eval()
print(f"ℹ️ Model ready | latent_dim={latent_dim}, d_model={d_model}, seq_len={seq_len}")

# ------------------------
# ✅ Add projection for mismatched dims
# ------------------------
latent_to_dmodel = torch.nn.Linear(latent_dim, d_model).to(device)

# ------------------------
# ✅ Generation function
# ------------------------
def generate_music(model, seq_len=512, latent_dim=128, d_model=4, temperature=0.8, device="cpu"):
    model.eval()
    with torch.no_grad():
        z = torch.randn(1, latent_dim).to(device) * temperature
        z_proj = latent_to_dmodel(z)
        z_seq = z_proj.unsqueeze(1).repeat(1, seq_len, 1)

        try:
            t_out = model.transformer(z_seq, z_seq)
        except Exception:
            # some transformers use (seq_len, batch, d_model)
            t_out = model.transformer(z_seq.permute(1,0,2), z_seq.permute(1,0,2)).permute(1,0,2)

        decoded = model.decoder(t_out).squeeze(0).cpu().numpy()
        return decoded

# ------------------------
# ✅ Convert decoded → notes → MIDI → WAV
# ------------------------
def decoded_to_notes(decoded, note_min=48, note_max=84, smooth=5):
    mel = decoded.mean(axis=1)
    if smooth > 1:
        kernel = np.ones(smooth)/smooth
        mel = np.convolve(mel, kernel, mode="same")
    mel = (mel - mel.min()) / (mel.max() - mel.min() + 1e-9)
    return np.round(note_min + mel * (note_max - note_min)).astype(int)

def notes_to_midi(notes, output_file="final_fixed.mid", step=0.2):
    midi = pretty_midi.PrettyMIDI()
    piano = pretty_midi.Instrument(program=0)
    start = 0.0
    for n in notes:
        end = start + step
        piano.notes.append(pretty_midi.Note(velocity=90, pitch=int(n), start=start, end=end))
        start = end
    midi.instruments.append(piano)
    midi.write(output_file)
    print(f"🎵 MIDI saved: {output_file}")
    return output_file

def midi_to_wav(midi_path, wav_path="final_fixed.wav"):
    try:
        midi_data = pretty_midi.PrettyMIDI(midi_path)
        audio = midi_data.fluidsynth()
        sf.write(wav_path, audio, 44100)
        print(f"🎧 WAV saved: {wav_path}")
    except Exception as e:
        print(f"⚠️ Could not convert to WAV: {e}")

# ------------------------
# 🚀 Generate rich piano melody
# ------------------------
decoded = generate_music(model, seq_len=seq_len, latent_dim=latent_dim, d_model=d_model, temperature=0.85, device=device)
notes = decoded_to_notes(decoded)
midi_file = notes_to_midi(notes, "final_rich_piano.mid", step=0.25)
midi_to_wav(midi_file, "final_rich_piano.wav")

print("\n✅ Done! Listen to 'final_rich_piano.mid' or 'final_rich_piano.wav'")


✅ Using device: cpu
✅ Model weights loaded successfully.
ℹ️ Model ready | latent_dim=128, d_model=256, seq_len=512
🎵 MIDI saved: final_rich_piano.mid
⚠️ Could not convert to WAV: module 'fluidsynth' has no attribute 'Synth'

✅ Done! Listen to 'final_rich_piano.mid' or 'final_rich_piano.wav'


In [5]:
# ============================================================
# 🎵 Enrich Generated Melody with Chords & Bass
# ============================================================
import pretty_midi
import numpy as np

def add_harmony_and_bass(midi_in_path, midi_out_path="final_rich_piano_harmony.mid",
                         chord_density=4, bass_octave_shift=-24):
    """
    Enriches a single-track melody MIDI file by adding:
    - Sustained chord harmonies (triads)
    - A bass track following chord roots

    Args:
        midi_in_path: Path to base melody MIDI file (e.g., 'final_rich_piano.mid')
        midi_out_path: Output multi-layer MIDI file
        chord_density: number of notes to group per chord (affects harmonic rhythm)
        bass_octave_shift: shift for bass notes (e.g., -24 -> two octaves down)
    """
    melody = pretty_midi.PrettyMIDI(midi_in_path)
    melody_track = melody.instruments[0]
    notes = sorted(melody_track.notes, key=lambda n: n.start)
    print(f"🎶 Melody has {len(notes)} notes — generating harmonies...")

    # Create new instruments
    harmony = pretty_midi.Instrument(program=0, name="Harmony Piano")
    bass = pretty_midi.Instrument(program=32, name="Acoustic Bass")  # GM 32 = bass

    # Group notes for harmony
    for i in range(0, len(notes), chord_density):
        group = notes[i:i+chord_density]
        if not group:
            continue
        start = group[0].start
        end = group[-1].end if len(group) > 1 else start + 0.4
        root_pitch = int(np.median([n.pitch for n in group]))

        # Build chord triad (root, +4, +7 semitones = major)
        chord = [root_pitch, root_pitch + 4, root_pitch + 7]

        for p in chord:
            harmony.notes.append(pretty_midi.Note(
                velocity=70, pitch=p, start=start, end=end
            ))

        # Bass note: root pitch shifted down
        bass_note = pretty_midi.Note(
            velocity=80, pitch=max(21, root_pitch + bass_octave_shift),
            start=start, end=end
        )
        bass.notes.append(bass_note)

    # Combine
    rich_midi = pretty_midi.PrettyMIDI()
    rich_midi.instruments.append(melody_track)
    rich_midi.instruments.append(harmony)
    rich_midi.instruments.append(bass)

    rich_midi.write(midi_out_path)
    print(f"✅ Rich harmony MIDI saved: {midi_out_path}")
    return midi_out_path


# ============================================================
# 🚀 Run Harmony Enhancement
# ============================================================
harmony_file = add_harmony_and_bass("final_rich_piano.mid", "final_rich_piano_harmony.mid")

# Optional: preview playable if pyfluidsynth installed
try:
    from IPython.display import Audio
    pm = pretty_midi.PrettyMIDI(harmony_file)
    audio = pm.fluidsynth()
    display(Audio(audio, rate=44100))
except Exception as e:
    print("⚠️ Audio playback skipped:", e)


🎶 Melody has 512 notes — generating harmonies...
✅ Rich harmony MIDI saved: final_rich_piano_harmony.mid
⚠️ Audio playback skipped: module 'fluidsynth' has no attribute 'Synth'


In [6]:
# ============================================================
# 🎵 Melody Beautifier: Scale, Rhythm, Expression, Harmony
# ============================================================
import pretty_midi
import numpy as np
import random

def beautify_music(
    midi_in="final_rich_piano.mid",
    midi_out="final_beautified_piano.mid",
    scale_type="major",
    key_root=60,  # C4
):
    # --- Load melody ---
    pm = pretty_midi.PrettyMIDI(midi_in)
    melody = pm.instruments[0]
    print(f"🎹 Loaded {len(melody.notes)} notes from {midi_in}")

    # --- Define scale ---
    major_scale = [0, 2, 4, 5, 7, 9, 11]
    minor_scale = [0, 2, 3, 5, 7, 8, 10]
    scale = major_scale if scale_type == "major" else minor_scale

    # --- Quantize notes to chosen scale ---
    def snap_to_scale(pitch):
        rel = (pitch - key_root) % 12
        closest = min(scale, key=lambda x: abs(x - rel))
        return pitch - rel + closest

    for n in melody.notes:
        n.pitch = snap_to_scale(n.pitch)

    # --- Add expression: dynamic velocity and rhythmic swing ---
    for i, n in enumerate(melody.notes):
        n.velocity = int(60 + 20 * np.sin(i / 3))  # natural phrasing
        if i % 5 == 0:
            n.end += 0.05  # slight sustain on downbeats

    # --- Add richer harmony (real progressions) ---
    harmony = pretty_midi.Instrument(program=0, name="Piano Harmony")
    bass = pretty_midi.Instrument(program=33, name="Bass")

    # Real chord progressions
    chord_progressions = [
        [0, 4, 7],    # C major
        [2, 5, 9],    # Dm
        [4, 7, 11],   # E minor
        [5, 9, 0],    # F major
        [7, 11, 2],   # G major
        [9, 0, 4]     # A minor
    ]

    chords = []
    for i in range(0, len(melody.notes), 8):
        root = random.choice(chord_progressions)
        start = melody.notes[i].start
        end = melody.notes[min(i + 7, len(melody.notes) - 1)].end
        for p in root:
            pitch = key_root + p
            harmony.notes.append(pretty_midi.Note(
                velocity=70, pitch=pitch, start=start, end=end
            ))
        # Bass root
        bass_note = key_root + root[0] - 24
        bass.notes.append(pretty_midi.Note(
            velocity=80, pitch=bass_note, start=start, end=end
        ))

    # --- Combine all ---
    beautified = pretty_midi.PrettyMIDI()
    beautified.instruments.append(melody)
    beautified.instruments.append(harmony)
    beautified.instruments.append(bass)
    beautified.write(midi_out)

    print(f"✅ Beautified, expressive MIDI saved: {midi_out}")
    return midi_out


# ============================================================
# 🚀 Run the beautifier
# ============================================================
final_beautified = beautify_music(
    midi_in="final_rich_piano.mid",
    midi_out="final_beautified_piano.mid",
    scale_type="major",  # change to "minor" for a darker tone
    key_root=60  # 60 = Middle C
)


🎹 Loaded 512 notes from final_rich_piano.mid
✅ Beautified, expressive MIDI saved: final_beautified_piano.mid


In [7]:
# ============================================================
# 🎹 Melody Beautifier v2 — Emotional & Melodic Piano Composer
# ============================================================
import pretty_midi
import numpy as np
import random

def beautify_music_v2(
    midi_in="final_rich_piano.mid",
    midi_out="final_beautified_piano_emotional.mid",
    mood="cinematic",  # "sad", "happy", "hopeful", "dreamy", "cinematic"
    key_root=60,  # C4
    base_bpm=80
):
    print(f"🎧 Beautifying {midi_in} → mood: {mood}")

    pm = pretty_midi.PrettyMIDI(midi_in)
    melody = pm.instruments[0]
    print(f"🎵 Loaded {len(melody.notes)} base melody notes")

    # --- Define scales ---
    major = [0, 2, 4, 5, 7, 9, 11]
    minor = [0, 2, 3, 5, 7, 8, 10]
    lydian = [0, 2, 4, 6, 7, 9, 11]
    dorian = [0, 2, 3, 5, 7, 9, 10]
    scale = major if mood in ["happy", "hopeful"] else minor

    # --- Chord Progressions by Mood ---
    chord_presets = {
        "happy": [[0, 4, 7], [7, 11, 2], [9, 0, 4], [5, 9, 0]],       # I–V–vi–IV
        "hopeful": [[0, 4, 7], [5, 9, 0], [9, 0, 4], [7, 11, 2]],     # I–IV–vi–V
        "sad": [[9, 0, 4], [5, 9, 0], [7, 11, 2], [0, 4, 7]],         # vi–IV–V–I
        "dreamy": [[0, 5, 9], [7, 11, 2], [4, 7, 11], [2, 5, 9]],     # sus-like chords
        "cinematic": [[0, 7, 10], [5, 9, 0], [7, 0, 4], [9, 0, 7]]    # open, emotional
    }
    chords = chord_presets.get(mood, chord_presets["cinematic"])

    # --- Quantize to Scale ---
    def snap_to_scale(p):
        rel = (p - key_root) % 12
        nearest = min(scale, key=lambda s: abs(s - rel))
        return p - rel + nearest

    for n in melody.notes:
        n.pitch = snap_to_scale(n.pitch)

    # --- Dynamic phrasing ---
    for i, n in enumerate(melody.notes):
        n.velocity = int(55 + 35 * np.sin(i / 4.0))
        if i % 6 == 0:
            n.end += 0.08  # emphasize phrasing
        n.start += random.uniform(-0.02, 0.02)
        n.end += random.uniform(-0.02, 0.03)

    # --- Build harmony + bass layers ---
    harmony = pretty_midi.Instrument(program=0, name="Harmony Piano")
    bass = pretty_midi.Instrument(program=33, name="Bass")

    notes_per_bar = 8
    bar_dur = 60.0 / base_bpm * 4
    total_bars = max(1, len(melody.notes) // notes_per_bar)

    for b in range(total_bars):
        chord = chords[b % len(chords)]
        start = melody.notes[b * notes_per_bar].start
        end = start + bar_dur * random.uniform(0.9, 1.2)

        # Harmony (mid register)
        for p in chord:
            harmony.notes.append(pretty_midi.Note(
                velocity=70 + random.randint(-10, 10),
                pitch=key_root + p,
                start=start,
                end=end
            ))

        # Bass
        bass_note = key_root + chord[0] - 24
        bass.notes.append(pretty_midi.Note(
            velocity=85,
            pitch=bass_note,
            start=start,
            end=end
        ))

    # --- Melody contour shaping ---
    for i, n in enumerate(melody.notes):
        # occasional octave jump for emotion
        if i % random.randint(12, 20) == 0:
            n.pitch += random.choice([-12, 0, 12])
        if mood in ["cinematic", "dreamy"]:
            n.velocity = int(n.velocity * random.uniform(0.8, 1.3))

    # --- Combine ---
    out_pm = pretty_midi.PrettyMIDI()
    out_pm.instruments.append(melody)
    out_pm.instruments.append(harmony)
    out_pm.instruments.append(bass)
    out_pm.write(midi_out)

    print(f"✅ Emotional piano saved: {midi_out}")
    return midi_out


# ============================================================
# 🚀 Run the upgraded beautifier
# ============================================================
final_beautified_v2 = beautify_music_v2(
    midi_in="final_rich_piano.mid",
    midi_out="final_beautified_piano_emotional.mid",
    mood="cinematic",   # try "sad", "hopeful", "dreamy"
    key_root=60,
    base_bpm=85
)


🎧 Beautifying final_rich_piano.mid → mood: cinematic
🎵 Loaded 512 base melody notes
✅ Emotional piano saved: final_beautified_piano_emotional.mid


In [8]:
# ============================================================
# 🎹 Melody Beautifier v3.1 — Emotionally Safe & Realistic Composer
# ============================================================
import pretty_midi
import numpy as np
import random

def beautify_music_v3_1(
    midi_in="final_rich_piano.mid",
    midi_out="final_emotional_piano_v3.mid",
    mood="cinematic",  # "sad", "hopeful", "dreamy", "cinematic", "peaceful"
    key_root=60,  # Middle C
    base_bpm=80
):
    print(f"🎧 Beautifying '{midi_in}' → mood: {mood}")

    pm = pretty_midi.PrettyMIDI(midi_in)
    melody = pm.instruments[0]
    print(f"🎵 Loaded {len(melody.notes)} melody notes")

    # --- Define scales ---
    scales = {
        "major": [0, 2, 4, 5, 7, 9, 11],
        "minor": [0, 2, 3, 5, 7, 8, 10],
        "dorian": [0, 2, 3, 5, 7, 9, 10],
        "lydian": [0, 2, 4, 6, 7, 9, 11]
    }

    scale_type = "minor" if mood in ["sad", "cinematic", "peaceful"] else "major"
    scale = scales[scale_type]

    # --- Chord Progressions ---
    chord_presets = {
        "sad": [[9, 0, 4], [5, 9, 0], [7, 11, 2], [0, 4, 7]],
        "hopeful": [[0, 4, 7], [5, 9, 0], [9, 0, 4], [7, 11, 2]],
        "dreamy": [[0, 5, 9], [4, 7, 11], [2, 5, 9], [7, 11, 2]],
        "cinematic": [[0, 7, 10], [5, 9, 0], [7, 0, 4], [9, 0, 7]],
        "peaceful": [[0, 4, 7], [2, 5, 9], [5, 9, 0], [0, 4, 7]]
    }
    chords = chord_presets[mood]

    # --- Scale quantization ---
    def snap_to_scale(p):
        rel = (p - key_root) % 12
        nearest = min(scale, key=lambda s: abs(s - rel))
        new_pitch = p - rel + nearest
        return max(21, min(108, new_pitch))  # safe piano range

    for n in melody.notes:
        n.pitch = snap_to_scale(n.pitch)

    # --- Expressive phrasing ---
    for i, n in enumerate(melody.notes):
        n.velocity = int(55 + 35 * np.sin(i / 5))
        n.start += random.uniform(-0.03, 0.03)
        n.end += random.uniform(-0.02, 0.05)
        if i % 8 == 0:
            n.end += 0.1  # natural sustain

    # --- Emotional velocity curve (crescendo & fade) ---
    total_notes = len(melody.notes)
    for i, n in enumerate(melody.notes):
        curve = np.sin((i / total_notes) * np.pi)
        n.velocity = int(max(30, min(120, n.velocity * (0.6 + 0.8 * curve))))

    # --- Micro tempo rubato ---
    for n in melody.notes:
        shift = np.sin(n.start * 0.7) * 0.02
        n.start += shift
        n.end += shift

    # --- Build harmony + bass layers ---
    harmony = pretty_midi.Instrument(program=0, name="Harmony Piano")
    bass = pretty_midi.Instrument(program=33, name="Bass")

    notes_per_bar = 8
    bar_dur = 60.0 / base_bpm * 4
    total_bars = max(1, len(melody.notes) // notes_per_bar)

    for b in range(total_bars):
        chord = chords[b % len(chords)]
        start = melody.notes[b * notes_per_bar].start
        end = start + bar_dur * random.uniform(0.9, 1.2)
        inversion = random.choice([0, 12, -12])

        for p in chord:
            pitch = key_root + p + inversion
            pitch = max(21, min(108, pitch))  # keep in piano range
            harmony.notes.append(pretty_midi.Note(
                velocity=random.randint(60, 80),
                pitch=pitch,
                start=start,
                end=end
            ))

        # Bass root
        bass_pitch = max(21, min(108, key_root + chord[0] - 24))
        bass_start = start + random.uniform(-0.1, 0.05)
        bass_end = end + random.uniform(0.05, 0.15)
        bass.notes.append(pretty_midi.Note(
            velocity=85,
            pitch=bass_pitch,
            start=bass_start,
            end=bass_end
        ))

    # --- Melody contour & emotional shaping ---
    for i, n in enumerate(melody.notes):
        if i % random.randint(10, 16) == 0:
            n.pitch += random.choice([-12, 0, 12])
            n.pitch = max(21, min(108, n.pitch))
        if mood in ["cinematic", "dreamy"]:
            n.velocity = int(min(120, n.velocity * random.uniform(0.9, 1.3)))

    # --- Combine ---
    out_pm = pretty_midi.PrettyMIDI()
    out_pm.instruments.append(melody)
    out_pm.instruments.append(harmony)
    out_pm.instruments.append(bass)
    out_pm.write(midi_out)

    print(f"✅ Saved expressive piano safely: {midi_out}")
    return midi_out


# ============================================================
# 🚀 Run the new stable version
# ============================================================
final_emotional_v3_1 = beautify_music_v3_1(
    midi_in="final_rich_piano.mid",
    midi_out="final_emotional_piano_v3_1.mid",
    mood="cinematic",   # Try "sad", "dreamy", "hopeful"
    key_root=60,
    base_bpm=80
)


🎧 Beautifying 'final_rich_piano.mid' → mood: cinematic
🎵 Loaded 512 melody notes
✅ Saved expressive piano safely: final_emotional_piano_v3_1.mid


In [9]:
# ============================================================
# 🎹 Professional Melodious & Emotional Music Generator v4.0
# ============================================================
import pretty_midi
import numpy as np
import random

def generate_melodious_music(
    midi_in="final_rich_piano.mid",
    midi_out="beautiful_emotional_piano.mid",
    mood="cinematic",
    key_root=60,
    base_bpm=72,
    scale_type="minor"
):
    """
    Enhanced music beautifier with professional-grade emotional expression
    
    Args:
        mood: "sad", "hopeful", "dreamy", "cinematic", "peaceful", "romantic"
        scale_type: "minor", "major", "pentatonic", "dorian", "phrygian"
    """
    print(f"🎧 Creating beautiful music → mood: {mood}, scale: {scale_type}")

    pm = pretty_midi.PrettyMIDI(midi_in)
    melody = pm.instruments[0]
    original_notes = len(melody.notes)
    print(f"🎵 Processing {original_notes} melody notes")

    # ═══════════════════════════════════════════════════════
    # SECTION 1: ENHANCED SCALE DEFINITIONS
    # ═══════════════════════════════════════════════════════
    scales = {
        "major": [0, 2, 4, 5, 7, 9, 11],
        "minor": [0, 2, 3, 5, 7, 8, 10],          # Natural minor
        "harmonic_minor": [0, 2, 3, 5, 7, 8, 11], # More dramatic
        "melodic_minor": [0, 2, 3, 5, 7, 9, 11],  # Ascending jazz
        "pentatonic": [0, 2, 4, 7, 9],            # Always pleasant
        "blues": [0, 3, 5, 6, 7, 10],             # Soulful
        "dorian": [0, 2, 3, 5, 7, 9, 10],         # Jazz/folk
        "phrygian": [0, 1, 3, 5, 7, 8, 10],       # Spanish/exotic
        "lydian": [0, 2, 4, 6, 7, 9, 11],         # Dreamy/floating
        "mixolydian": [0, 2, 4, 5, 7, 9, 10]      # Rock/folk
    }

    # Auto-select best scale for mood
    scale_map = {
        "sad": "harmonic_minor",
        "hopeful": "major",
        "dreamy": "lydian",
        "cinematic": "melodic_minor",
        "peaceful": "pentatonic",
        "romantic": "major"
    }
    
    if scale_type in scales:
        scale = scales[scale_type]
    else:
        scale = scales[scale_map.get(mood, "minor")]

    # ═══════════════════════════════════════════════════════
    # SECTION 2: SOPHISTICATED CHORD PROGRESSIONS
    # ═══════════════════════════════════════════════════════
    chord_progressions = {
        "sad": [
            [0, 3, 7],    # i minor
            [7, 10, 2],   # V minor
            [5, 8, 0],    # iv minor
            [3, 7, 10]    # III major
        ],
        "hopeful": [
            [0, 4, 7],    # I major
            [5, 9, 0],    # IV major
            [7, 11, 2],   # V major
            [9, 0, 4]     # vi minor
        ],
        "dreamy": [
            [0, 4, 7, 11], # Imaj7
            [5, 9, 0, 4],  # IVmaj7
            [2, 5, 9, 0],  # ii7
            [7, 11, 2, 5]  # Vmaj7
        ],
        "cinematic": [
            [0, 3, 7, 10], # i7
            [5, 8, 0, 3],  # iv7
            [7, 10, 2, 5], # V7
            [3, 7, 10, 2]  # VImaj7
        ],
        "peaceful": [
            [0, 4, 7],     # I
            [2, 5, 9],     # ii
            [5, 9, 0],     # IV
            [0, 4, 7]      # I
        ],
        "romantic": [
            [0, 4, 7, 11], # Imaj7
            [9, 0, 4, 7],  # vi7
            [2, 5, 9, 0],  # ii7
            [7, 11, 2, 5]  # V7
        ]
    }
    
    chords = chord_progressions.get(mood, chord_progressions["peaceful"])

    # ═══════════════════════════════════════════════════════
    # SECTION 3: INTELLIGENT SCALE QUANTIZATION
    # ═══════════════════════════════════════════════════════
    def snap_to_scale(pitch, prefer_direction=0):
        """Smart scale quantization with musical direction preference"""
        rel = (pitch - key_root) % 12
        
        # Find nearest scale degree
        distances = [(s, abs(s - rel)) for s in scale]
        distances.sort(key=lambda x: x[1])
        
        # Prefer upward or downward motion if specified
        if prefer_direction != 0 and len(distances) > 1:
            if prefer_direction > 0:  # Prefer ascending
                scale_note = max(distances[0][0], distances[1][0])
            else:  # Prefer descending
                scale_note = min(distances[0][0], distances[1][0])
        else:
            scale_note = distances[0][0]
        
        new_pitch = pitch - rel + scale_note
        return max(36, min(96, new_pitch))  # Professional piano range

    # ═══════════════════════════════════════════════════════
    # SECTION 4: MELODIC CONTOUR SHAPING
    # ═══════════════════════════════════════════════════════
    # Create smooth, flowing melodies with natural arcs
    melody.notes.sort(key=lambda x: x.start)
    
    for i, note in enumerate(melody.notes):
        # Determine melodic direction
        if i > 0:
            prev_pitch = melody.notes[i-1].pitch
            direction = 1 if note.pitch > prev_pitch else -1
        else:
            direction = 0
            
        note.pitch = snap_to_scale(note.pitch, direction)
        
        # Add melodic shape: arch form (low → high → low)
        progress = i / max(1, len(melody.notes) - 1)
        arch = np.sin(progress * np.pi)  # Natural rise and fall
        
        # Gentle pitch adjustments for melodic flow
        if random.random() < 0.15:  # 15% chance of octave jump
            note.pitch += int(12 * arch * random.choice([-1, 1]))
            note.pitch = snap_to_scale(note.pitch)

    # ═══════════════════════════════════════════════════════
    # SECTION 5: EXPRESSIVE DYNAMICS & PHRASING
    # ═══════════════════════════════════════════════════════
    total_notes = len(melody.notes)
    
    for i, note in enumerate(melody.notes):
        progress = i / max(1, total_notes - 1)
        
        # Multi-layered dynamic curve
        # 1. Overall emotional arc (crescendo → climax → fade)
        emotional_curve = np.sin(progress * np.pi) ** 0.7
        
        # 2. Phrase-level dynamics (8-bar phrases)
        phrase_position = (i % 32) / 32
        phrase_curve = 0.7 + 0.3 * np.sin(phrase_position * np.pi)
        
        # 3. Micro-variations (human feel)
        micro_variation = random.uniform(0.85, 1.15)
        
        # Combine dynamics
        base_velocity = 50 if mood in ["peaceful", "dreamy"] else 65
        dynamic_range = 55 if mood == "cinematic" else 45
        
        velocity = base_velocity + dynamic_range * emotional_curve * phrase_curve * micro_variation
        note.velocity = int(np.clip(velocity, 30, 115))
        
        # Climax emphasis
        if 0.45 < progress < 0.65:  # Climax region
            note.velocity = int(min(120, note.velocity * 1.2))

    # ═══════════════════════════════════════════════════════
    # SECTION 6: NATURAL TIMING & RUBATO
    # ═══════════════════════════════════════════════════════
    for i, note in enumerate(melody.notes):
        progress = i / max(1, total_notes - 1)
        
        # Phrase endings: slow down (ritardando)
        if i % 16 in [14, 15]:
            time_stretch = 1.15
        # Phrase beginnings: slightly faster
        elif i % 16 == 0:
            time_stretch = 0.95
        else:
            time_stretch = 1.0
        
        # Micro-timing (human imperfection)
        humanization = random.uniform(-0.02, 0.02)
        note.start += humanization
        
        # Natural note lengths
        if i % 8 in [3, 7]:  # Phrase breaths
            note.end = note.start + (note.end - note.start) * 1.3
        else:
            note.end = note.start + (note.end - note.start) * time_stretch
        
        # Add breathing space between phrases
        if (i + 1) % 16 == 0 and i < total_notes - 1:
            melody.notes[i + 1].start += 0.1

    # ═══════════════════════════════════════════════════════
    # SECTION 7: SOPHISTICATED HARMONY LAYER
    # ═══════════════════════════════════════════════════════
    harmony = pretty_midi.Instrument(program=0, name="Harmony")
    
    # Calculate musical timing
    beat_duration = 60.0 / base_bpm
    bar_duration = beat_duration * 4
    
    # Group melody into bars
    if melody.notes:
        song_duration = melody.notes[-1].end
        num_bars = int(np.ceil(song_duration / bar_duration))
        
        for bar in range(num_bars):
            chord = chords[bar % len(chords)]
            bar_start = bar * bar_duration
            bar_end = bar_start + bar_duration
            
            # Find melody notes in this bar for voice leading
            bar_notes = [n for n in melody.notes if bar_start <= n.start < bar_end]
            if not bar_notes:
                continue
            
            # Determine chord voicing (inversion)
            avg_melody_pitch = np.mean([n.pitch for n in bar_notes])
            
            # Keep harmony below or around melody
            if avg_melody_pitch > 72:  # High melody
                voicing_offset = -12
            elif avg_melody_pitch < 60:  # Low melody
                voicing_offset = 0
            else:
                voicing_offset = -7
            
            # Create chord with smooth voice leading
            for note_offset in chord:
                pitch = key_root + note_offset + voicing_offset
                pitch = max(36, min(84, pitch))
                
                # Arpeggiate for dreamy/peaceful moods
                if mood in ["dreamy", "peaceful"]:
                    arp_delay = chord.index(note_offset) * 0.05
                else:
                    arp_delay = 0
                
                harmony.notes.append(pretty_midi.Note(
                    velocity=random.randint(45, 65),
                    pitch=pitch,
                    start=bar_start + arp_delay,
                    end=bar_end + random.uniform(-0.1, 0.1)
                ))

    # ═══════════════════════════════════════════════════════
    # SECTION 8: EXPRESSIVE BASS LINE
    # ═══════════════════════════════════════════════════════
    bass = pretty_midi.Instrument(program=32, name="Acoustic Bass")
    
    for bar in range(num_bars):
        chord = chords[bar % len(chords)]
        bar_start = bar * bar_duration
        
        # Root note on beat 1
        bass_root = key_root + chord[0] - 24
        bass_root = max(28, min(52, bass_root))
        
        bass.notes.append(pretty_midi.Note(
            velocity=75,
            pitch=bass_root,
            start=bar_start,
            end=bar_start + beat_duration * 2.5
        ))
        
        # Fifth on beat 3 (walking bass)
        if len(chord) >= 2 and mood not in ["peaceful", "dreamy"]:
            bass_fifth = key_root + chord[1] - 24
            bass_fifth = max(28, min(52, bass_fifth))
            
            bass.notes.append(pretty_midi.Note(
                velocity=65,
                pitch=bass_fifth,
                start=bar_start + beat_duration * 2,
                end=bar_start + beat_duration * 3.5
            ))

    # ═══════════════════════════════════════════════════════
    # SECTION 9: ADD SUBTLE EMBELLISHMENTS
    # ═══════════════════════════════════════════════════════
    embellishment = pretty_midi.Instrument(program=0, name="Embellishment")
    
    # Add occasional grace notes and trills
    for i in range(0, len(melody.notes), random.randint(12, 20)):
        if i >= len(melody.notes):
            break
            
        note = melody.notes[i]
        
        # Grace note before main note
        if random.random() < 0.3:
            grace_pitch = note.pitch + random.choice([2, -2])
            grace_pitch = snap_to_scale(grace_pitch)
            
            embellishment.notes.append(pretty_midi.Note(
                velocity=note.velocity - 15,
                pitch=grace_pitch,
                start=note.start - 0.08,
                end=note.start
            ))

    # ═══════════════════════════════════════════════════════
    # SECTION 10: FINAL ASSEMBLY & EXPORT
    # ═══════════════════════════════════════════════════════
    output = pretty_midi.PrettyMIDI()
    output.instruments.append(melody)
    output.instruments.append(harmony)
    output.instruments.append(bass)
    
    if len(embellishment.notes) > 0:
        output.instruments.append(embellishment)
    
    output.write(midi_out)
    
    print(f"✅ Created beautiful music: {midi_out}")
    print(f"📊 Statistics:")
    print(f"   • Melody: {len(melody.notes)} notes")
    print(f"   • Harmony: {len(harmony.notes)} notes")
    print(f"   • Bass: {len(bass.notes)} notes")
    print(f"   • Total duration: {output.get_end_time():.1f}s")
    
    return midi_out


# ============================================================
# 🚀 USAGE EXAMPLES
# ============================================================

if __name__ == "__main__":
    # Example 1: Cinematic emotional piece
    generate_melodious_music(
        midi_in="final_rich_piano.mid",
        midi_out="cinematic_masterpiece.mid",
        mood="cinematic",
        key_root=60,  # C
        base_bpm=72,
        scale_type="melodic_minor"
    )
    
    # Example 2: Peaceful meditation
    # generate_melodious_music(
    #     midi_in="final_rich_piano.mid",
    #     midi_out="peaceful_meditation.mid",
    #     mood="peaceful",
    #     key_root=65,  # F
    #     base_bpm=60,
    #     scale_type="pentatonic"
    # )
    
    # Example 3: Romantic ballad
    # generate_melodious_music(
    #     midi_in="final_rich_piano.mid",
    #     midi_out="romantic_ballad.mid",
    #     mood="romantic",
    #     key_root=62,  # D
    #     base_bpm=66,
    #     scale_type="major"
    # )

🎧 Creating beautiful music → mood: cinematic, scale: melodic_minor
🎵 Processing 512 melody notes


✅ Created beautiful music: cinematic_masterpiece.mid
📊 Statistics:
   • Melody: 512 notes
   • Harmony: 156 notes
   • Bass: 78 notes
   • Total duration: 130.1s


In [10]:
# ============================================================
# 🎬 CINEMATIC PIANO COMPOSER - Film Score Quality
# Every detail matters: from breath to emotion to storytelling
# ============================================================
import pretty_midi
import numpy as np
import random
from copy import deepcopy

def create_cinematic_piano(
    midi_in="final_rich_piano.mid",
    midi_out="movie_quality_piano.mid",
    emotion="bittersweet",  # "melancholy", "hopeful", "bittersweet", "triumphant", "nostalgic", "romantic", "epic"
    key_root=60,
    tempo=68,
    story_arc="hero_journey"  # "hero_journey", "love_story", "loss_and_hope", "mystery", "gentle_dawn"
):
    """
    Creates movie-quality piano music with every subtle detail:
    - Emotional breathing and phrasing
    - Storytelling arc (beginning, development, climax, resolution)
    - Human imperfections that make it alive
    - Orchestral thinking applied to piano
    - Every note has purpose and feeling
    """
    
    print(f"🎬 Creating cinematic piano: {emotion} | Story: {story_arc}")
    
    pm = pretty_midi.PrettyMIDI(midi_in)
    melody_raw = pm.instruments[0]
    
    # ═══════════════════════════════════════════════════════
    # 🎭 EMOTIONAL DNA - The Soul of the Music
    # ═══════════════════════════════════════════════════════
    emotional_profiles = {
        "melancholy": {
            "scale": [0, 2, 3, 5, 7, 8, 11],  # Harmonic minor - hauntingly beautiful
            "chords": [[0,3,7], [8,11,3], [5,8,0], [7,11,2], [3,7,10], [0,3,7]],
            "velocity_base": 45, "velocity_range": 40,
            "tempo_variation": 0.25,  # More rubato = more emotion
            "sustain_factor": 1.4,    # Let notes ring, like memories
            "silence_weight": 0.3,    # Silence speaks volumes
            "touch": "tender"
        },
        "hopeful": {
            "scale": [0, 2, 4, 5, 7, 9, 11],  # Major - warm and lifting
            "chords": [[0,4,7], [5,9,0], [9,0,4], [2,5,9], [7,11,2], [0,4,7,11]],
            "velocity_base": 60, "velocity_range": 50,
            "tempo_variation": 0.15,
            "sustain_factor": 1.2,
            "silence_weight": 0.15,
            "touch": "bright"
        },
        "bittersweet": {
            "scale": [0, 2, 3, 5, 7, 9, 10],  # Dorian - sweet sadness
            "chords": [[0,3,7], [5,9,0], [7,10,2], [3,7,10], [5,9,0], [0,3,7,10]],
            "velocity_base": 55, "velocity_range": 45,
            "tempo_variation": 0.20,
            "sustain_factor": 1.3,
            "silence_weight": 0.25,
            "touch": "longing"
        },
        "triumphant": {
            "scale": [0, 2, 4, 5, 7, 9, 11],
            "chords": [[0,4,7], [7,11,2], [9,0,4], [0,4,7,11], [5,9,0,4], [0,4,7]],
            "velocity_base": 75, "velocity_range": 45,
            "tempo_variation": 0.10,
            "sustain_factor": 1.1,
            "silence_weight": 0.05,
            "touch": "powerful"
        },
        "nostalgic": {
            "scale": [0, 2, 4, 6, 7, 9, 11],  # Lydian - dreamy past
            "chords": [[0,4,7,11], [2,6,9], [5,9,0,4], [7,11,2,6], [0,4,7]],
            "velocity_base": 48, "velocity_range": 38,
            "tempo_variation": 0.28,
            "sustain_factor": 1.5,
            "silence_weight": 0.35,
            "touch": "delicate"
        },
        "romantic": {
            "scale": [0, 2, 4, 5, 7, 9, 11],
            "chords": [[0,4,7,11], [9,0,4,7], [5,9,0,4], [2,5,9,0], [7,11,2,5], [0,4,7,11]],
            "velocity_base": 52, "velocity_range": 42,
            "tempo_variation": 0.22,
            "sustain_factor": 1.35,
            "silence_weight": 0.20,
            "touch": "intimate"
        },
        "epic": {
            "scale": [0, 2, 3, 5, 7, 8, 10],  # Natural minor - grand
            "chords": [[0,3,7,10], [5,8,0,3], [7,10,2,5], [3,7,10,2], [8,0,3,7], [0,3,7]],
            "velocity_base": 70, "velocity_range": 50,
            "tempo_variation": 0.12,
            "sustain_factor": 1.25,
            "silence_weight": 0.10,
            "touch": "majestic"
        }
    }
    
    profile = emotional_profiles.get(emotion, emotional_profiles["bittersweet"])
    scale = profile["scale"]
    
    # ═══════════════════════════════════════════════════════
    # 📖 STORY STRUCTURE - Every movie has acts
    # ═══════════════════════════════════════════════════════
    story_structures = {
        "hero_journey": {
            # Soft beginning → challenge → triumph → peaceful ending
            "acts": [
                {"weight": 0.2, "intensity": 0.3, "density": 0.6, "register": "mid"},    # Humble beginning
                {"weight": 0.3, "intensity": 0.7, "density": 0.8, "register": "rising"}, # Challenge
                {"weight": 0.3, "intensity": 1.0, "density": 1.0, "register": "high"},   # Climax/Victory
                {"weight": 0.2, "intensity": 0.4, "density": 0.5, "register": "mid"}     # Resolution
            ]
        },
        "love_story": {
            # Meeting → growing → passion → tender forever
            "acts": [
                {"weight": 0.25, "intensity": 0.4, "density": 0.5, "register": "mid"},
                {"weight": 0.25, "intensity": 0.6, "density": 0.7, "register": "mid-high"},
                {"weight": 0.25, "intensity": 0.9, "density": 0.9, "register": "high"},
                {"weight": 0.25, "intensity": 0.5, "density": 0.6, "register": "mid"}
            ]
        },
        "loss_and_hope": {
            # Pain → deepening → darkest moment → finding light
            "acts": [
                {"weight": 0.3, "intensity": 0.6, "density": 0.7, "register": "mid"},
                {"weight": 0.3, "intensity": 0.8, "density": 0.8, "register": "low"},
                {"weight": 0.2, "intensity": 0.9, "density": 0.9, "register": "low"},
                {"weight": 0.2, "intensity": 0.5, "density": 0.6, "register": "rising"}
            ]
        },
        "mystery": {
            # Question → investigation → revelation → understanding
            "acts": [
                {"weight": 0.25, "intensity": 0.4, "density": 0.5, "register": "low-mid"},
                {"weight": 0.35, "intensity": 0.6, "density": 0.7, "register": "mid"},
                {"weight": 0.25, "intensity": 0.9, "density": 0.9, "register": "high"},
                {"weight": 0.15, "intensity": 0.4, "density": 0.5, "register": "mid"}
            ]
        },
        "gentle_dawn": {
            # Darkness → first light → sunrise → full day
            "acts": [
                {"weight": 0.3, "intensity": 0.2, "density": 0.4, "register": "low"},
                {"weight": 0.3, "intensity": 0.5, "density": 0.6, "register": "mid"},
                {"weight": 0.25, "intensity": 0.7, "density": 0.8, "register": "mid-high"},
                {"weight": 0.15, "intensity": 0.6, "density": 0.7, "register": "mid-high"}
            ]
        }
    }
    
    story = story_structures.get(story_arc, story_structures["hero_journey"])
    
    # ═══════════════════════════════════════════════════════
    # 🎹 PIANO PHYSICS & HUMAN TOUCH
    # ═══════════════════════════════════════════════════════
    
    def apply_piano_physics(note, touch_type="tender"):
        """
        Real piano behavior:
        - Harder attack = brighter tone (higher velocity)
        - Softer attack = warmer tone (lower velocity)  
        - Key release matters (pedal simulation)
        - Finger weight and gravity
        """
        
        touch_profiles = {
            "tender": {"attack": 0.85, "decay": 1.3, "brightness": 0.8},
            "bright": {"attack": 1.1, "decay": 1.0, "brightness": 1.2},
            "longing": {"attack": 0.9, "decay": 1.4, "brightness": 0.9},
            "powerful": {"attack": 1.3, "decay": 0.9, "brightness": 1.3},
            "delicate": {"attack": 0.75, "decay": 1.5, "brightness": 0.7},
            "intimate": {"attack": 0.8, "decay": 1.35, "brightness": 0.85},
            "majestic": {"attack": 1.2, "decay": 1.1, "brightness": 1.15}
        }
        
        touch = touch_profiles.get(touch_type, touch_profiles["tender"])
        
        # Velocity (how hard the key is struck)
        note.velocity = int(np.clip(note.velocity * touch["attack"], 20, 127))
        
        # Duration (sustain pedal effect)
        duration = note.end - note.start
        note.end = note.start + (duration * touch["decay"])
        
        # Micro-timing (human imperfection - fingers aren't machines)
        note.start += random.gauss(0, 0.008)  # ±8ms human timing variance
        
        return note
    
    def snap_to_scale_musical(pitch, scale, root, prefer_leap=False):
        """Not just quantize - think musically"""
        rel = (pitch - root) % 12
        
        # Find closest scale degrees
        options = []
        for s in scale:
            dist = abs(s - rel)
            if dist > 6:  # Wrap around octave
                dist = 12 - dist
            options.append((s, dist))
        
        options.sort(key=lambda x: x[1])
        
        # Musical preference: sometimes leap to create interest
        if prefer_leap and len(options) > 1 and random.random() < 0.3:
            chosen = options[1][0]  # Take 2nd closest for variety
        else:
            chosen = options[0][0]
        
        new_pitch = pitch - rel + chosen
        
        # Keep in expressive piano range (not too high, not too low)
        return int(np.clip(new_pitch, 36, 96))
    
    # ═══════════════════════════════════════════════════════
    # 🎼 MELODIC INTELLIGENCE - Make it Singable
    # ═══════════════════════════════════════════════════════
    
    melody_notes = sorted(melody_raw.notes, key=lambda x: x.start)
    total_notes = len(melody_notes)
    
    # Assign each note to a story act
    act_boundaries = []
    cumulative = 0
    for act in story["acts"]:
        cumulative += act["weight"]
        act_boundaries.append(int(cumulative * total_notes))
    
    current_act = 0
    
    for i, note in enumerate(melody_notes):
        # Determine which act we're in
        while current_act < len(act_boundaries) - 1 and i >= act_boundaries[current_act]:
            current_act += 1
        
        act = story["acts"][current_act]
        progress_in_act = (i - (act_boundaries[current_act-1] if current_act > 0 else 0)) / max(1, act_boundaries[current_act] - (act_boundaries[current_act-1] if current_act > 0 else 0))
        
        # ══════════ MELODIC CONTOUR ══════════
        # Create memorable, hummable melodies
        
        # Determine melodic direction preference
        if i > 0:
            prev_pitch = melody_notes[i-1].pitch
            interval = note.pitch - prev_pitch
            
            # Avoid awkward jumps (more than octave)
            if abs(interval) > 12:
                note.pitch = prev_pitch + np.sign(interval) * random.randint(3, 7)
            
            # Create contour: steps and skips, not just steps
            if abs(interval) <= 2 and random.random() < 0.2:  # Too stepwise? Add skip
                note.pitch += random.choice([-5, -4, 4, 5])
        
        # Quantize musically
        prefer_leap = (i % 8) in [0, 4]  # Start of phrases can leap
        note.pitch = snap_to_scale_musical(note.pitch, scale, key_root, prefer_leap)
        
        # ══════════ REGISTER (HEIGHT) CONTROL ══════════
        register = act["register"]
        
        register_targets = {
            "low": (45, 60),
            "mid": (55, 70),
            "low-mid": (50, 65),
            "mid-high": (60, 75),
            "high": (65, 80),
            "rising": (55 + int(20 * progress_in_act), 70 + int(20 * progress_in_act))
        }
        
        target_range = register_targets.get(register, (55, 70))
        
        # Gently guide pitch to target register
        if note.pitch < target_range[0]:
            note.pitch += 12
        elif note.pitch > target_range[1]:
            note.pitch -= 12
        
        note.pitch = int(np.clip(note.pitch, 36, 96))
        
        # ══════════ DYNAMICS (LOUDNESS) ══════════
        # Multi-layered expression
        
        # 1. Story act intensity
        base_vel = profile["velocity_base"] + (profile["velocity_range"] * act["intensity"])
        
        # 2. Phrase-level shaping (natural breathing in 4-8 bar phrases)
        phrase_length = random.randint(16, 32)
        phrase_pos = (i % phrase_length) / phrase_length
        phrase_shape = np.sin(phrase_pos * np.pi) ** 0.6  # Arch shape
        
        # 3. Note-level micro-dynamics (human touch)
        micro_dyn = random.gauss(1.0, 0.12)
        
        # 4. Emotional accents (stress important notes)
        accent = 1.0
        if i % 8 == 0:  # Downbeat
            accent = 1.15
        elif (i % 8) in [3, 5, 7]:  # Syncopation points
            accent = random.choice([0.9, 1.1])
        
        velocity = base_vel * phrase_shape * micro_dyn * accent
        note.velocity = int(np.clip(velocity, 20, 127))
        
        # ══════════ TIMING & RHYTHM ══════════
        # Not mechanical - human feel
        
        # Rubato (tempo flexibility) - more at phrase ends
        if (i % 16) >= 14:  # End of phrase
            time_stretch = 1.0 + profile["tempo_variation"]
            note.end = note.start + (note.end - note.start) * time_stretch
        elif (i % 16) == 0:  # Start of phrase - slight push
            time_stretch = 0.95
            note.end = note.start + (note.end - note.start) * time_stretch
        
        # Natural sustain and release
        duration = note.end - note.start
        note.end = note.start + (duration * profile["sustain_factor"])
        
        # Apply piano physics
        note = apply_piano_physics(note, profile["touch"])
        
        # ══════════ SILENCE & SPACE ══════════
        # Pauses are part of the music
        if (i % 16) == 15 and random.random() < profile["silence_weight"]:
            if i < len(melody_notes) - 1:
                silence_gap = random.uniform(0.15, 0.4)
                melody_notes[i+1].start += silence_gap
    
    # ═══════════════════════════════════════════════════════
    # 🎻 HARMONY - The Orchestra Behind the Piano
    # ═══════════════════════════════════════════════════════
    
    chords = profile["chords"]
    harmony_layer = pretty_midi.Instrument(program=0, name="Harmony")
    strings_layer = pretty_midi.Instrument(program=48, name="Strings")  # String ensemble feel
    
    # Calculate musical time
    beat_dur = 60.0 / tempo
    measure_dur = beat_dur * 4
    
    if melody_notes:
        duration = melody_notes[-1].end
        num_measures = int(np.ceil(duration / measure_dur))
        
        for measure in range(num_measures):
            # Determine act for this measure
            measure_progress = measure / max(1, num_measures)
            act_idx = min(len(story["acts"]) - 1, int(measure_progress / (1.0 / len(story["acts"]))))
            act = story["acts"][act_idx]
            
            chord = chords[measure % len(chords)]
            m_start = measure * measure_dur
            m_end = m_start + measure_dur
            
            # Find melody in this measure to determine harmony voicing
            measure_melody = [n for n in melody_notes if m_start <= n.start < m_end]
            
            if measure_melody:
                avg_melody = np.mean([n.pitch for n in measure_melody])
                
                # Voice leading: keep harmony below or interweaving with melody
                if avg_melody > 72:
                    voicing = -12  # Lower voicing
                elif avg_melody < 54:
                    voicing = 12   # Higher voicing
                else:
                    voicing = -7   # Interweave
                
                # Chord density based on story intensity
                num_voices = min(len(chord), int(2 + act["density"] * 2))
                
                # Create chord tones
                for j, interval in enumerate(chord[:num_voices]):
                    pitch = key_root + interval + voicing
                    pitch = int(np.clip(pitch, 36, 84))
                    
                    # Arpeggiate for texture (not block chords)
                    arp_delay = j * 0.04 if act["density"] < 0.8 else j * 0.02
                    
                    # Harmony dynamics follow story
                    harm_vel = int(35 + 40 * act["intensity"])
                    harm_vel += random.randint(-5, 5)
                    
                    harmony_layer.notes.append(pretty_midi.Note(
                        velocity=harm_vel,
                        pitch=pitch,
                        start=m_start + arp_delay,
                        end=m_end + random.uniform(-0.1, 0.15)
                    ))
                    
                    # Add string pad for emotional depth (sparse)
                    if j == 0 and measure % 2 == 0:  # Root note only, every other measure
                        strings_layer.notes.append(pretty_midi.Note(
                            velocity=int(25 + 20 * act["intensity"]),
                            pitch=pitch,
                            start=m_start,
                            end=m_end + measure_dur * 0.3  # Overlap into next
                        ))
    
    # ═══════════════════════════════════════════════════════
    # 🎸 BASS - The Foundation
    # ═══════════════════════════════════════════════════════
    
    bass = pretty_midi.Instrument(program=32, name="Acoustic Bass")
    
    for measure in range(num_measures):
        measure_progress = measure / max(1, num_measures)
        act_idx = min(len(story["acts"]) - 1, int(measure_progress / (1.0 / len(story["acts"]))))
        act = story["acts"][act_idx]
        
        chord = chords[measure % len(chords)]
        m_start = measure * measure_dur
        
        # Root on downbeat
        bass_root = key_root + chord[0] - 24
        bass_root = int(np.clip(bass_root, 28, 52))
        
        bass_vel = int(55 + 35 * act["intensity"])
        
        bass.notes.append(pretty_midi.Note(
            velocity=bass_vel,
            pitch=bass_root,
            start=m_start,
            end=m_start + beat_dur * 3.5
        ))
        
        # Walking bass (fifth or third) on beat 3
        if act["density"] > 0.6 and len(chord) >= 2:
            bass_walk = key_root + chord[1] - 24
            bass_walk = int(np.clip(bass_walk, 28, 52))
            
            bass.notes.append(pretty_midi.Note(
                velocity=bass_vel - 10,
                pitch=bass_walk,
                start=m_start + beat_dur * 2.5,
                end=m_start + beat_dur * 3.8
            ))
    
    # ═══════════════════════════════════════════════════════
    # ✨ SPARKLE LAYER - High register magic
    # ═══════════════════════════════════════════════════════
    
    sparkle = pretty_midi.Instrument(program=11, name="Music Box")  # Vibraphone/celesta feel
    
    # Add delicate high notes at emotional peaks
    for i, note in enumerate(melody_notes):
        progress = i / max(1, total_notes - 1)
        
        # Only in climax regions and occasionally
        if 0.5 < progress < 0.75 and random.random() < 0.15:
            sparkle_pitch = note.pitch + 24  # Two octaves up
            sparkle_pitch = int(np.clip(sparkle_pitch, 84, 108))
            
            sparkle.notes.append(pretty_midi.Note(
                velocity=random.randint(35, 55),
                pitch=sparkle_pitch,
                start=note.start + random.uniform(0, 0.05),
                end=note.start + random.uniform(0.3, 0.6)
            ))
    
    # ═══════════════════════════════════════════════════════
    # 🎬 FINAL MIX & MASTER
    # ═══════════════════════════════════════════════════════
    
    # Create new MIDI
    output = pretty_midi.PrettyMIDI(initial_tempo=tempo)
    
    # Rename and organize
    melody_raw.name = "Piano Melody (Lead)"
    melody_raw.notes = melody_notes
    
    # Layer order matters for mixing
    output.instruments.append(bass)              # Foundation
    output.instruments.append(strings_layer)     # Pad
    output.instruments.append(harmony_layer)     # Harmony
    output.instruments.append(melody_raw)        # Lead
    if len(sparkle.notes) > 5:
        output.instruments.append(sparkle)       # Sparkle on top
    
    output.write(midi_out)
    
    # ═══════════════════════════════════════════════════════
    # 📊 COMPOSITION REPORT
    # ═══════════════════════════════════════════════════════
    
    print(f"\n{'='*60}")
    print(f"🎬 CINEMATIC PIANO COMPOSITION COMPLETE")
    print(f"{'='*60}")
    print(f"🎭 Emotion: {emotion.upper()}")
    print(f"📖 Story: {story_arc.replace('_', ' ').title()}")
    print(f"🎹 Key: {['C','C#','D','D#','E','F','F#','G','G#','A','A#','B'][key_root%12]}")
    print(f"⏱️  Tempo: {tempo} BPM")
    print(f"⏲️  Duration: {output.get_end_time():.1f}s ({output.get_end_time()/60:.1f} min)")
    print(f"\n📊 Layers:")
    print(f"   🎹 Melody: {len(melody_raw.notes)} notes")
    print(f"   🎻 Harmony: {len(harmony_layer.notes)} notes")
    print(f"   🎸 Bass: {len(bass.notes)} notes")
    print(f"   🎼 Strings: {len(strings_layer.notes)} notes")
    if len(sparkle.notes) > 0:
        print(f"   ✨ Sparkle: {len(sparkle.notes)} notes")
    print(f"\n💝 This music tells a story and breathes like a living thing.")
    print(f"{'='*60}\n")
    
    return midi_out


# ============================================================
# 🎬 MOVIE SCENE EXAMPLES
# ============================================================

if __name__ == "__main__":
    
    # # Scene 1: Opening credits - mysterious beginning
    # create_cinematic_piano(
    #     midi_in="final_rich_piano.mid",
    #     midi_out="01_opening_mysterious.mid",
    #     emotion="nostalgic",
    #     story_arc="mystery",
    #     key_root=57,  # A minor
    #     tempo=66
    # )
    
    # Scene 2: Love theme - two characters meet
    create_cinematic_piano(
        midi_in="final_rich_piano.mid",
        midi_out="02_love_theme.mid",
        emotion="romantic",
        story_arc="love_story",
        key_root=62,  # D major
        tempo=70
    )
    
    # Scene 3: Hero's struggle and victory
    # create_cinematic_piano(
    #     midi_in="final_rich_piano.mid",
    #     midi_out="03_hero_triumph.mid",
    #     emotion="triumphant",
    #     story_arc="hero_journey",
    #     key_root=60,  # C major
    #     tempo=76
    # )
    
    # Scene 4: Sad goodbye / loss scene
    # create_cinematic_piano(
    #     midi_in="final_rich_piano.mid",
    #     midi_out="04_goodbye.mid",
    #     emotion="melancholy",
    #     story_arc="loss_and_hope",
    #     key_root=55,  # G minor
    #     tempo=58
    # )
    
    # Scene 5: Ending - bittersweet resolution
    # create_cinematic_piano(
    #     midi_in="final_rich_piano.mid",
    #     midi_out="05_ending_bittersweet.mid",
    #     emotion="bittersweet",
    #     story_arc="gentle_dawn",
    #     key_root=65,  # F major
    #     tempo=64
    # )

🎬 Creating cinematic piano: romantic | Story: love_story



🎬 CINEMATIC PIANO COMPOSITION COMPLETE
🎭 Emotion: ROMANTIC
📖 Story: Love Story
🎹 Key: D
⏱️  Tempo: 70 BPM
⏲️  Duration: 130.4s (2.2 min)

📊 Layers:
   🎹 Melody: 512 notes
   🎻 Harmony: 114 notes
   🎸 Bass: 57 notes
   🎼 Strings: 19 notes
   ✨ Sparkle: 18 notes

💝 This music tells a story and breathes like a living thing.



In [11]:
pip install react

Note: you may need to restart the kernel to use updated packages.


In [12]:
# ============================================================
# 🎬 IMPROVED CINEMATIC PIANO COMPOSER
# NEW FEATURES:
# ✅ Clean Mode - No overlapping notes
# ✅ Better mixing - Each layer has proper space
# ✅ Smoother transitions
# ✅ Configurable layer control
# ============================================================

import pretty_midi
import numpy as np
import random
from copy import deepcopy

def create_cinematic_piano(
    midi_in="final_rich_piano.mid",
    midi_out="movie_quality_piano.mid",
    emotion="bittersweet",
    key_root=60,
    tempo=68,
    story_arc="hero_journey",
    # NEW PARAMETERS
    clean_mode=True,           # Prevents overlaps
    dynamic_range="moderate",   # "low", "moderate", "high"
    phrase_length="medium",     # "short", "medium", "long"
    enable_layers=None          # Dict to enable/disable layers
):
    """
    Enhanced cinematic piano with clean, professional output.
    
    NEW FEATURES:
    - clean_mode: Intelligently prevents note overlaps
    - dynamic_range: Controls volume contrast
    - phrase_length: Adjusts musical breathing
    - enable_layers: Choose which instruments to include
    """
    
    if enable_layers is None:
        enable_layers = {
            "melody": True,
            "harmony": True,
            "bass": True,
            "strings": True,
            "sparkle": True
        }
    
    print(f"🎬 Creating cinematic piano: {emotion} | Story: {story_arc}")
    print(f"🎛️  Clean Mode: {clean_mode} | Dynamic Range: {dynamic_range}")
    
    pm = pretty_midi.PrettyMIDI(midi_in)
    melody_raw = pm.instruments[0]
    
    # ═══════════════════════════════════════════════════════
    # 🎭 EMOTIONAL DNA (Same as before)
    # ═══════════════════════════════════════════════════════
    
    emotional_profiles = {
        "melancholy": {
            "scale": [0, 2, 3, 5, 7, 8, 11],
            "chords": [[0,3,7], [8,11,3], [5,8,0], [7,11,2], [3,7,10], [0,3,7]],
            "velocity_base": 45, "velocity_range": 40,
            "tempo_variation": 0.25,
            "sustain_factor": 1.4,
            "silence_weight": 0.3,
            "touch": "tender"
        },
        "hopeful": {
            "scale": [0, 2, 4, 5, 7, 9, 11],
            "chords": [[0,4,7], [5,9,0], [9,0,4], [2,5,9], [7,11,2], [0,4,7,11]],
            "velocity_base": 60, "velocity_range": 50,
            "tempo_variation": 0.15,
            "sustain_factor": 1.2,
            "silence_weight": 0.15,
            "touch": "bright"
        },
        "bittersweet": {
            "scale": [0, 2, 3, 5, 7, 9, 10],
            "chords": [[0,3,7], [5,9,0], [7,10,2], [3,7,10], [5,9,0], [0,3,7,10]],
            "velocity_base": 55, "velocity_range": 45,
            "tempo_variation": 0.20,
            "sustain_factor": 1.3,
            "silence_weight": 0.25,
            "touch": "longing"
        },
        "triumphant": {
            "scale": [0, 2, 4, 5, 7, 9, 11],
            "chords": [[0,4,7], [7,11,2], [9,0,4], [0,4,7,11], [5,9,0,4], [0,4,7]],
            "velocity_base": 75, "velocity_range": 45,
            "tempo_variation": 0.10,
            "sustain_factor": 1.1,
            "silence_weight": 0.05,
            "touch": "powerful"
        },
        "nostalgic": {
            "scale": [0, 2, 4, 6, 7, 9, 11],
            "chords": [[0,4,7,11], [2,6,9], [5,9,0,4], [7,11,2,6], [0,4,7]],
            "velocity_base": 48, "velocity_range": 38,
            "tempo_variation": 0.28,
            "sustain_factor": 1.5,
            "silence_weight": 0.35,
            "touch": "delicate"
        },
        "romantic": {
            "scale": [0, 2, 4, 5, 7, 9, 11],
            "chords": [[0,4,7,11], [9,0,4,7], [5,9,0,4], [2,5,9,0], [7,11,2,5], [0,4,7,11]],
            "velocity_base": 52, "velocity_range": 42,
            "tempo_variation": 0.22,
            "sustain_factor": 1.35,
            "silence_weight": 0.20,
            "touch": "intimate"
        },
        "epic": {
            "scale": [0, 2, 3, 5, 7, 8, 10],
            "chords": [[0,3,7,10], [5,8,0,3], [7,10,2,5], [3,7,10,2], [8,0,3,7], [0,3,7]],
            "velocity_base": 70, "velocity_range": 50,
            "tempo_variation": 0.12,
            "sustain_factor": 1.25,
            "silence_weight": 0.10,
            "touch": "majestic"
        }
    }
    
    profile = emotional_profiles.get(emotion, emotional_profiles["bittersweet"])
    scale = profile["scale"]
    
    # ═══════════════════════════════════════════════════════
    # 📊 DYNAMIC RANGE CONTROL (NEW)
    # ═══════════════════════════════════════════════════════
    
    dynamic_multipliers = {
        "low": {"base": 0.85, "range": 0.7},      # Subtle, gentle
        "moderate": {"base": 1.0, "range": 1.0},  # Balanced
        "high": {"base": 1.15, "range": 1.3}      # Dramatic, expressive
    }
    
    dyn_mult = dynamic_multipliers.get(dynamic_range, dynamic_multipliers["moderate"])
    profile["velocity_base"] *= dyn_mult["base"]
    profile["velocity_range"] *= dyn_mult["range"]
    
    # ═══════════════════════════════════════════════════════
    # 🎵 PHRASE LENGTH CONTROL (NEW)
    # ═══════════════════════════════════════════════════════
    
    phrase_configs = {
        "short": {"min": 8, "max": 16, "breath": 0.3},    # Intimate, conversational
        "medium": {"min": 16, "max": 32, "breath": 0.25}, # Balanced
        "long": {"min": 24, "max": 48, "breath": 0.2}     # Epic, sweeping
    }
    
    phrase_cfg = phrase_configs.get(phrase_length, phrase_configs["medium"])
    
    # ═══════════════════════════════════════════════════════
    # 📖 STORY STRUCTURE (Same as before)
    # ═══════════════════════════════════════════════════════
    
    story_structures = {
        "hero_journey": {
            "acts": [
                {"weight": 0.2, "intensity": 0.3, "density": 0.6, "register": "mid"},
                {"weight": 0.3, "intensity": 0.7, "density": 0.8, "register": "rising"},
                {"weight": 0.3, "intensity": 1.0, "density": 1.0, "register": "high"},
                {"weight": 0.2, "intensity": 0.4, "density": 0.5, "register": "mid"}
            ]
        },
        "love_story": {
            "acts": [
                {"weight": 0.25, "intensity": 0.4, "density": 0.5, "register": "mid"},
                {"weight": 0.25, "intensity": 0.6, "density": 0.7, "register": "mid-high"},
                {"weight": 0.25, "intensity": 0.9, "density": 0.9, "register": "high"},
                {"weight": 0.25, "intensity": 0.5, "density": 0.6, "register": "mid"}
            ]
        },
        "loss_and_hope": {
            "acts": [
                {"weight": 0.3, "intensity": 0.6, "density": 0.7, "register": "mid"},
                {"weight": 0.3, "intensity": 0.8, "density": 0.8, "register": "low"},
                {"weight": 0.2, "intensity": 0.9, "density": 0.9, "register": "low"},
                {"weight": 0.2, "intensity": 0.5, "density": 0.6, "register": "rising"}
            ]
        },
        "mystery": {
            "acts": [
                {"weight": 0.25, "intensity": 0.4, "density": 0.5, "register": "low-mid"},
                {"weight": 0.35, "intensity": 0.6, "density": 0.7, "register": "mid"},
                {"weight": 0.25, "intensity": 0.9, "density": 0.9, "register": "high"},
                {"weight": 0.15, "intensity": 0.4, "density": 0.5, "register": "mid"}
            ]
        },
        "gentle_dawn": {
            "acts": [
                {"weight": 0.3, "intensity": 0.2, "density": 0.4, "register": "low"},
                {"weight": 0.3, "intensity": 0.5, "density": 0.6, "register": "mid"},
                {"weight": 0.25, "intensity": 0.7, "density": 0.8, "register": "mid-high"},
                {"weight": 0.15, "intensity": 0.6, "density": 0.7, "register": "mid-high"}
            ]
        }
    }
    
    story = story_structures.get(story_arc, story_structures["hero_journey"])
    
    # ═══════════════════════════════════════════════════════
    # 🎹 HELPER FUNCTIONS
    # ═══════════════════════════════════════════════════════
    
    def apply_piano_physics(note, touch_type="tender"):
        touch_profiles = {
            "tender": {"attack": 0.85, "decay": 1.3, "brightness": 0.8},
            "bright": {"attack": 1.1, "decay": 1.0, "brightness": 1.2},
            "longing": {"attack": 0.9, "decay": 1.4, "brightness": 0.9},
            "powerful": {"attack": 1.3, "decay": 0.9, "brightness": 1.3},
            "delicate": {"attack": 0.75, "decay": 1.5, "brightness": 0.7},
            "intimate": {"attack": 0.8, "decay": 1.35, "brightness": 0.85},
            "majestic": {"attack": 1.2, "decay": 1.1, "brightness": 1.15}
        }
        
        touch = touch_profiles.get(touch_type, touch_profiles["tender"])
        note.velocity = int(np.clip(note.velocity * touch["attack"], 20, 127))
        duration = note.end - note.start
        note.end = note.start + (duration * touch["decay"])
        note.start += random.gauss(0, 0.008)
        return note
    
    def snap_to_scale_musical(pitch, scale, root, prefer_leap=False):
        rel = (pitch - root) % 12
        options = []
        for s in scale:
            dist = abs(s - rel)
            if dist > 6:
                dist = 12 - dist
            options.append((s, dist))
        options.sort(key=lambda x: x[1])
        
        if prefer_leap and len(options) > 1 and random.random() < 0.3:
            chosen = options[1][0]
        else:
            chosen = options[0][0]
        
        new_pitch = pitch - rel + chosen
        return int(np.clip(new_pitch, 36, 96))
    
    def clean_overlaps(notes):
        """NEW: Remove overlapping notes for cleaner sound"""
        if not clean_mode or len(notes) == 0:
            return notes
        
        notes = sorted(notes, key=lambda x: x.start)
        cleaned = []
        
        for i, note in enumerate(notes):
            if i > 0:
                prev = cleaned[-1]
                # If overlap detected, trim previous note
                if prev.end > note.start:
                    gap = 0.02  # 20ms gap for clarity
                    prev.end = max(prev.start + 0.1, note.start - gap)
            cleaned.append(note)
        
        return cleaned
    
    # ═══════════════════════════════════════════════════════
    # 🎼 PROCESS MELODY
    # ═══════════════════════════════════════════════════════
    
    melody_notes = sorted(melody_raw.notes, key=lambda x: x.start)
    total_notes = len(melody_notes)
    
    act_boundaries = []
    cumulative = 0
    for act in story["acts"]:
        cumulative += act["weight"]
        act_boundaries.append(int(cumulative * total_notes))
    
    current_act = 0
    
    for i, note in enumerate(melody_notes):
        while current_act < len(act_boundaries) - 1 and i >= act_boundaries[current_act]:
            current_act += 1
        
        act = story["acts"][current_act]
        progress_in_act = (i - (act_boundaries[current_act-1] if current_act > 0 else 0)) / max(1, act_boundaries[current_act] - (act_boundaries[current_act-1] if current_act > 0 else 0))
        
        # Melodic processing
        if i > 0:
            prev_pitch = melody_notes[i-1].pitch
            interval = note.pitch - prev_pitch
            if abs(interval) > 12:
                note.pitch = prev_pitch + np.sign(interval) * random.randint(3, 7)
            if abs(interval) <= 2 and random.random() < 0.2:
                note.pitch += random.choice([-5, -4, 4, 5])
        
        prefer_leap = (i % 8) in [0, 4]
        note.pitch = snap_to_scale_musical(note.pitch, scale, key_root, prefer_leap)
        
        # Register control
        register = act["register"]
        register_targets = {
            "low": (45, 60), "mid": (55, 70), "low-mid": (50, 65),
            "mid-high": (60, 75), "high": (65, 80),
            "rising": (55 + int(20 * progress_in_act), 70 + int(20 * progress_in_act))
        }
        target_range = register_targets.get(register, (55, 70))
        
        if note.pitch < target_range[0]:
            note.pitch += 12
        elif note.pitch > target_range[1]:
            note.pitch -= 12
        note.pitch = int(np.clip(note.pitch, 36, 96))
        
        # Dynamics
        base_vel = profile["velocity_base"] + (profile["velocity_range"] * act["intensity"])
        phrase_len = random.randint(phrase_cfg["min"], phrase_cfg["max"])
        phrase_pos = (i % phrase_len) / phrase_len
        phrase_shape = np.sin(phrase_pos * np.pi) ** 0.6
        micro_dyn = random.gauss(1.0, 0.12)
        accent = 1.15 if i % 8 == 0 else (random.choice([0.9, 1.1]) if (i % 8) in [3, 5, 7] else 1.0)
        
        velocity = base_vel * phrase_shape * micro_dyn * accent
        note.velocity = int(np.clip(velocity, 20, 127))
        
        # Timing
        if (i % 16) >= 14:
            time_stretch = 1.0 + profile["tempo_variation"]
            note.end = note.start + (note.end - note.start) * time_stretch
        elif (i % 16) == 0:
            time_stretch = 0.95
            note.end = note.start + (note.end - note.start) * time_stretch
        
        duration = note.end - note.start
        note.end = note.start + (duration * profile["sustain_factor"])
        note = apply_piano_physics(note, profile["touch"])
        
        # Silence
        if (i % 16) == 15 and random.random() < (profile["silence_weight"] * phrase_cfg["breath"]):
            if i < len(melody_notes) - 1:
                silence_gap = random.uniform(0.15, 0.4)
                melody_notes[i+1].start += silence_gap
    
    # Clean melody overlaps
    melody_notes = clean_overlaps(melody_notes)
    
    # ═══════════════════════════════════════════════════════
    # 🎻 CREATE LAYERS (Only if enabled)
    # ═══════════════════════════════════════════════════════
    
    chords = profile["chords"]
    beat_dur = 60.0 / tempo
    measure_dur = beat_dur * 4
    
    harmony_layer = pretty_midi.Instrument(program=0, name="Harmony") if enable_layers["harmony"] else None
    strings_layer = pretty_midi.Instrument(program=48, name="Strings") if enable_layers["strings"] else None
    bass = pretty_midi.Instrument(program=32, name="Bass") if enable_layers["bass"] else None
    sparkle = pretty_midi.Instrument(program=11, name="Sparkle") if enable_layers["sparkle"] else None
    
    if melody_notes:
        duration = melody_notes[-1].end
        num_measures = int(np.ceil(duration / measure_dur))
        
        for measure in range(num_measures):
            measure_progress = measure / max(1, num_measures)
            act_idx = min(len(story["acts"]) - 1, int(measure_progress / (1.0 / len(story["acts"]))))
            act = story["acts"][act_idx]
            
            chord = chords[measure % len(chords)]
            m_start = measure * measure_dur
            m_end = m_start + measure_dur
            
            measure_melody = [n for n in melody_notes if m_start <= n.start < m_end]
            
            if measure_melody:
                avg_melody = np.mean([n.pitch for n in measure_melody])
                voicing = -12 if avg_melody > 72 else (12 if avg_melody < 54 else -7)
                num_voices = min(len(chord), int(2 + act["density"] * 2))
                
                # Harmony
                if harmony_layer:
                    for j, interval in enumerate(chord[:num_voices]):
                        pitch = int(np.clip(key_root + interval + voicing, 36, 84))
                        arp_delay = j * 0.04 if act["density"] < 0.8 else j * 0.02
                        harm_vel = int(35 + 40 * act["intensity"]) + random.randint(-5, 5)
                        
                        harmony_layer.notes.append(pretty_midi.Note(
                            velocity=harm_vel, pitch=pitch,
                            start=m_start + arp_delay,
                            end=m_end + random.uniform(-0.1, 0.15)
                        ))
                
                # Strings
                if strings_layer and j == 0 and measure % 2 == 0:
                    strings_layer.notes.append(pretty_midi.Note(
                        velocity=int(25 + 20 * act["intensity"]),
                        pitch=pitch,
                        start=m_start,
                        end=m_end + measure_dur * 0.3
                    ))
                
                # Bass
                if bass:
                    bass_root = int(np.clip(key_root + chord[0] - 24, 28, 52))
                    bass_vel = int(55 + 35 * act["intensity"])
                    
                    bass.notes.append(pretty_midi.Note(
                        velocity=bass_vel, pitch=bass_root,
                        start=m_start, end=m_start + beat_dur * 3.5
                    ))
                    
                    if act["density"] > 0.6 and len(chord) >= 2:
                        bass_walk = int(np.clip(key_root + chord[1] - 24, 28, 52))
                        bass.notes.append(pretty_midi.Note(
                            velocity=bass_vel - 10, pitch=bass_walk,
                            start=m_start + beat_dur * 2.5,
                            end=m_start + beat_dur * 3.8
                        ))
        
        # Sparkle
        if sparkle:
            for i, note in enumerate(melody_notes):
                progress = i / max(1, total_notes - 1)
                if 0.5 < progress < 0.75 and random.random() < 0.15:
                    sparkle_pitch = int(np.clip(note.pitch + 24, 84, 108))
                    sparkle.notes.append(pretty_midi.Note(
                        velocity=random.randint(35, 55), pitch=sparkle_pitch,
                        start=note.start + random.uniform(0, 0.05),
                        end=note.start + random.uniform(0.3, 0.6)
                    ))
    
    # Clean all layers
    if harmony_layer:
        harmony_layer.notes = clean_overlaps(harmony_layer.notes)
    if bass:
        bass.notes = clean_overlaps(bass.notes)
    if strings_layer:
        strings_layer.notes = clean_overlaps(strings_layer.notes)
    
    # ═══════════════════════════════════════════════════════
    # 🎬 FINAL ASSEMBLY
    # ═══════════════════════════════════════════════════════
    
    output = pretty_midi.PrettyMIDI(initial_tempo=tempo)
    melody_raw.name = "Piano Melody"
    melody_raw.notes = melody_notes
    
    if bass and enable_layers["bass"]:
        output.instruments.append(bass)
    if strings_layer and enable_layers["strings"]:
        output.instruments.append(strings_layer)
    if harmony_layer and enable_layers["harmony"]:
        output.instruments.append(harmony_layer)
    output.instruments.append(melody_raw)
    if sparkle and enable_layers["sparkle"] and len(sparkle.notes) > 5:
        output.instruments.append(sparkle)
    
    output.write(midi_out)
    
    # Report
    print(f"\n{'='*60}")
    print(f"🎬 COMPOSITION COMPLETE")
    print(f"{'='*60}")
    print(f"Duration: {output.get_end_time():.1f}s")
    print(f"Melody: {len(melody_raw.notes)} notes")
    if harmony_layer:
        print(f"Harmony: {len(harmony_layer.notes)} notes")
    if bass:
        print(f"Bass: {len(bass.notes)} notes")
    print(f"✅ Clean mode {'ENABLED' if clean_mode else 'DISABLED'}")
    print(f"{'='*60}\n")
    
    return midi_out


# ============================================================
# 🎮 USAGE EXAMPLES
# ============================================================

if __name__ == "__main__":
    # Example 1: Clean romantic piece
    create_cinematic_piano(
        midi_in="final_rich_piano.mid",
        midi_out="romantic_clean.mid",
        emotion="romantic",
        story_arc="love_story",
        key_root=62,
        tempo=70,
        clean_mode=True,
        dynamic_range="moderate",
        phrase_length="medium"
    )
    
    # Example 2: Epic with only melody and bass
    # create_cinematic_piano(
    #     midi_in="final_rich_piano.mid",
    #     midi_out="epic_minimal.mid",
    #     emotion="epic",
    #     story_arc="hero_journey",
    #     key_root=60,
    #     tempo=80,
    #     clean_mode=True,
    #     dynamic_range="high",
    #     phrase_length="long",
    #     enable_layers={
    #         "melody": True,
    #         "harmony": False,
    #         "bass": True,
    #         "strings": False,
    #         "sparkle": False
    #     }
    # )

🎬 Creating cinematic piano: romantic | Story: love_story
🎛️  Clean Mode: True | Dynamic Range: moderate



🎬 COMPOSITION COMPLETE
Duration: 130.4s
Melody: 512 notes
Harmony: 114 notes
Bass: 57 notes
✅ Clean mode ENABLED



In [13]:
# ============================================================
# 🎬 CINEMATIC PIANO COMPOSER - JUPYTER NOTEBOOK VERSION
# Interactive UI with ipywidgets + Full Music Generation
# ============================================================

import pretty_midi
import numpy as np
import random
from copy import deepcopy
import ipywidgets as widgets
from IPython.display import display, Audio, HTML, clear_output
import io

# ============================================================
# 🎼 MUSIC GENERATION ENGINE
# ============================================================

def create_cinematic_piano(
    midi_in="final_rich_piano.mid",
    midi_out="movie_quality_piano.mid",
    emotion="bittersweet",
    key_root=60,
    tempo=68,
    story_arc="hero_journey",
    clean_mode=True,
    dynamic_range="moderate",
    phrase_length="medium",
    enable_layers=None
):
    """
    Enhanced cinematic piano with clean, professional output.
    """
    
    if enable_layers is None:
        enable_layers = {
            "melody": True,
            "harmony": True,
            "bass": True,
            "strings": True,
            "sparkle": True
        }
    
    print(f"🎬 Creating: {emotion} | Story: {story_arc}")
    print(f"🎛️  Clean: {clean_mode} | Dynamics: {dynamic_range} | Key: {key_root}")
    
    try:
        pm = pretty_midi.PrettyMIDI(midi_in)
    except:
        print("⚠️ Input file not found. Creating from scratch...")
        pm = pretty_midi.PrettyMIDI()
        instrument = pretty_midi.Instrument(program=0)
        # Create simple melody
        for i in range(32):
            note = pretty_midi.Note(
                velocity=70,
                pitch=60 + random.choice([0, 2, 4, 5, 7, 9, 11]),
                start=i * 0.5,
                end=(i + 1) * 0.5
            )
            instrument.notes.append(note)
        pm.instruments.append(instrument)
    
    melody_raw = pm.instruments[0]
    
    # Emotional profiles
    emotional_profiles = {
        "melancholy": {
            "scale": [0, 2, 3, 5, 7, 8, 11],
            "chords": [[0,3,7], [8,11,3], [5,8,0], [7,11,2], [3,7,10], [0,3,7]],
            "velocity_base": 45, "velocity_range": 40,
            "tempo_variation": 0.25,
            "sustain_factor": 1.4,
            "silence_weight": 0.3,
            "touch": "tender"
        },
        "hopeful": {
            "scale": [0, 2, 4, 5, 7, 9, 11],
            "chords": [[0,4,7], [5,9,0], [9,0,4], [2,5,9], [7,11,2], [0,4,7,11]],
            "velocity_base": 60, "velocity_range": 50,
            "tempo_variation": 0.15,
            "sustain_factor": 1.2,
            "silence_weight": 0.15,
            "touch": "bright"
        },
        "bittersweet": {
            "scale": [0, 2, 3, 5, 7, 9, 10],
            "chords": [[0,3,7], [5,9,0], [7,10,2], [3,7,10], [5,9,0], [0,3,7,10]],
            "velocity_base": 55, "velocity_range": 45,
            "tempo_variation": 0.20,
            "sustain_factor": 1.3,
            "silence_weight": 0.25,
            "touch": "longing"
        },
        "triumphant": {
            "scale": [0, 2, 4, 5, 7, 9, 11],
            "chords": [[0,4,7], [7,11,2], [9,0,4], [0,4,7,11], [5,9,0,4], [0,4,7]],
            "velocity_base": 75, "velocity_range": 45,
            "tempo_variation": 0.10,
            "sustain_factor": 1.1,
            "silence_weight": 0.05,
            "touch": "powerful"
        },
        "nostalgic": {
            "scale": [0, 2, 4, 6, 7, 9, 11],
            "chords": [[0,4,7,11], [2,6,9], [5,9,0,4], [7,11,2,6], [0,4,7]],
            "velocity_base": 48, "velocity_range": 38,
            "tempo_variation": 0.28,
            "sustain_factor": 1.5,
            "silence_weight": 0.35,
            "touch": "delicate"
        },
        "romantic": {
            "scale": [0, 2, 4, 5, 7, 9, 11],
            "chords": [[0,4,7,11], [9,0,4,7], [5,9,0,4], [2,5,9,0], [7,11,2,5], [0,4,7,11]],
            "velocity_base": 52, "velocity_range": 42,
            "tempo_variation": 0.22,
            "sustain_factor": 1.35,
            "silence_weight": 0.20,
            "touch": "intimate"
        },
        "epic": {
            "scale": [0, 2, 3, 5, 7, 8, 10],
            "chords": [[0,3,7,10], [5,8,0,3], [7,10,2,5], [3,7,10,2], [8,0,3,7], [0,3,7]],
            "velocity_base": 70, "velocity_range": 50,
            "tempo_variation": 0.12,
            "sustain_factor": 1.25,
            "silence_weight": 0.10,
            "touch": "majestic"
        }
    }
    
    profile = emotional_profiles.get(emotion, emotional_profiles["bittersweet"])
    scale = profile["scale"]
    
    # Dynamic range adjustment
    dynamic_multipliers = {
        "low": {"base": 0.85, "range": 0.7},
        "moderate": {"base": 1.0, "range": 1.0},
        "high": {"base": 1.15, "range": 1.3}
    }
    dyn_mult = dynamic_multipliers.get(dynamic_range, dynamic_multipliers["moderate"])
    profile["velocity_base"] *= dyn_mult["base"]
    profile["velocity_range"] *= dyn_mult["range"]
    
    # Phrase length
    phrase_configs = {
        "short": {"min": 8, "max": 16, "breath": 0.3},
        "medium": {"min": 16, "max": 32, "breath": 0.25},
        "long": {"min": 24, "max": 48, "breath": 0.2}
    }
    phrase_cfg = phrase_configs.get(phrase_length, phrase_configs["medium"])
    
    # Story structures
    story_structures = {
        "hero_journey": {
            "acts": [
                {"weight": 0.2, "intensity": 0.3, "density": 0.6, "register": "mid"},
                {"weight": 0.3, "intensity": 0.7, "density": 0.8, "register": "rising"},
                {"weight": 0.3, "intensity": 1.0, "density": 1.0, "register": "high"},
                {"weight": 0.2, "intensity": 0.4, "density": 0.5, "register": "mid"}
            ]
        },
        "love_story": {
            "acts": [
                {"weight": 0.25, "intensity": 0.4, "density": 0.5, "register": "mid"},
                {"weight": 0.25, "intensity": 0.6, "density": 0.7, "register": "mid-high"},
                {"weight": 0.25, "intensity": 0.9, "density": 0.9, "register": "high"},
                {"weight": 0.25, "intensity": 0.5, "density": 0.6, "register": "mid"}
            ]
        },
        "loss_and_hope": {
            "acts": [
                {"weight": 0.3, "intensity": 0.6, "density": 0.7, "register": "mid"},
                {"weight": 0.3, "intensity": 0.8, "density": 0.8, "register": "low"},
                {"weight": 0.2, "intensity": 0.9, "density": 0.9, "register": "low"},
                {"weight": 0.2, "intensity": 0.5, "density": 0.6, "register": "rising"}
            ]
        },
        "mystery": {
            "acts": [
                {"weight": 0.25, "intensity": 0.4, "density": 0.5, "register": "low-mid"},
                {"weight": 0.35, "intensity": 0.6, "density": 0.7, "register": "mid"},
                {"weight": 0.25, "intensity": 0.9, "density": 0.9, "register": "high"},
                {"weight": 0.15, "intensity": 0.4, "density": 0.5, "register": "mid"}
            ]
        },
        "gentle_dawn": {
            "acts": [
                {"weight": 0.3, "intensity": 0.2, "density": 0.4, "register": "low"},
                {"weight": 0.3, "intensity": 0.5, "density": 0.6, "register": "mid"},
                {"weight": 0.25, "intensity": 0.7, "density": 0.8, "register": "mid-high"},
                {"weight": 0.15, "intensity": 0.6, "density": 0.7, "register": "mid-high"}
            ]
        }
    }
    story = story_structures.get(story_arc, story_structures["hero_journey"])
    
    # Helper functions
    def apply_piano_physics(note, touch_type="tender"):
        touch_profiles = {
            "tender": {"attack": 0.85, "decay": 1.3},
            "bright": {"attack": 1.1, "decay": 1.0},
            "longing": {"attack": 0.9, "decay": 1.4},
            "powerful": {"attack": 1.3, "decay": 0.9},
            "delicate": {"attack": 0.75, "decay": 1.5},
            "intimate": {"attack": 0.8, "decay": 1.35},
            "majestic": {"attack": 1.2, "decay": 1.1}
        }
        touch = touch_profiles.get(touch_type, touch_profiles["tender"])
        note.velocity = int(np.clip(note.velocity * touch["attack"], 20, 127))
        duration = note.end - note.start
        note.end = note.start + (duration * touch["decay"])
        note.start += random.gauss(0, 0.008)
        return note
    
    def snap_to_scale_musical(pitch, scale, root, prefer_leap=False):
        rel = (pitch - root) % 12
        options = []
        for s in scale:
            dist = abs(s - rel)
            if dist > 6:
                dist = 12 - dist
            options.append((s, dist))
        options.sort(key=lambda x: x[1])
        
        if prefer_leap and len(options) > 1 and random.random() < 0.3:
            chosen = options[1][0]
        else:
            chosen = options[0][0]
        
        new_pitch = pitch - rel + chosen
        return int(np.clip(new_pitch, 36, 96))
    
    def clean_overlaps(notes):
        if not clean_mode or len(notes) == 0:
            return notes
        notes = sorted(notes, key=lambda x: x.start)
        cleaned = []
        for i, note in enumerate(notes):
            if i > 0:
                prev = cleaned[-1]
                if prev.end > note.start:
                    gap = 0.02
                    prev.end = max(prev.start + 0.1, note.start - gap)
            cleaned.append(note)
        return cleaned
    
    # Process melody
    melody_notes = sorted(melody_raw.notes, key=lambda x: x.start)
    total_notes = len(melody_notes)
    
    act_boundaries = []
    cumulative = 0
    for act in story["acts"]:
        cumulative += act["weight"]
        act_boundaries.append(int(cumulative * total_notes))
    
    current_act = 0
    
    for i, note in enumerate(melody_notes):
        while current_act < len(act_boundaries) - 1 and i >= act_boundaries[current_act]:
            current_act += 1
        
        act = story["acts"][current_act]
        
        if i > 0:
            prev_pitch = melody_notes[i-1].pitch
            interval = note.pitch - prev_pitch
            if abs(interval) > 12:
                note.pitch = prev_pitch + np.sign(interval) * random.randint(3, 7)
            if abs(interval) <= 2 and random.random() < 0.2:
                note.pitch += random.choice([-5, -4, 4, 5])
        
        prefer_leap = (i % 8) in [0, 4]
        note.pitch = snap_to_scale_musical(note.pitch, scale, key_root, prefer_leap)
        
        register = act["register"]
        register_targets = {
            "low": (45, 60), "mid": (55, 70), "low-mid": (50, 65),
            "mid-high": (60, 75), "high": (65, 80),
            "rising": (55 + int(20 * (i/max(1,total_notes))), 70 + int(20 * (i/max(1,total_notes))))
        }
        target_range = register_targets.get(register, (55, 70))
        
        if note.pitch < target_range[0]:
            note.pitch += 12
        elif note.pitch > target_range[1]:
            note.pitch -= 12
        note.pitch = int(np.clip(note.pitch, 36, 96))
        
        base_vel = profile["velocity_base"] + (profile["velocity_range"] * act["intensity"])
        phrase_len = random.randint(phrase_cfg["min"], phrase_cfg["max"])
        phrase_pos = (i % phrase_len) / phrase_len
        phrase_shape = np.sin(phrase_pos * np.pi) ** 0.6
        micro_dyn = random.gauss(1.0, 0.12)
        accent = 1.15 if i % 8 == 0 else 1.0
        
        velocity = base_vel * phrase_shape * micro_dyn * accent
        note.velocity = int(np.clip(velocity, 20, 127))
        
        duration = note.end - note.start
        note.end = note.start + (duration * profile["sustain_factor"])
        note = apply_piano_physics(note, profile["touch"])
    
    melody_notes = clean_overlaps(melody_notes)
    
    # Create layers
    chords = profile["chords"]
    beat_dur = 60.0 / tempo
    measure_dur = beat_dur * 4
    
    harmony_layer = pretty_midi.Instrument(program=0, name="Harmony") if enable_layers.get("harmony") else None
    strings_layer = pretty_midi.Instrument(program=48, name="Strings") if enable_layers.get("strings") else None
    bass = pretty_midi.Instrument(program=32, name="Bass") if enable_layers.get("bass") else None
    sparkle = pretty_midi.Instrument(program=11, name="Sparkle") if enable_layers.get("sparkle") else None
    
    if melody_notes:
        duration = melody_notes[-1].end
        num_measures = int(np.ceil(duration / measure_dur))
        
        for measure in range(num_measures):
            measure_progress = measure / max(1, num_measures)
            act_idx = min(len(story["acts"]) - 1, int(measure_progress / (1.0 / len(story["acts"]))))
            act = story["acts"][act_idx]
            
            chord = chords[measure % len(chords)]
            m_start = measure * measure_dur
            m_end = m_start + measure_dur
            
            measure_melody = [n for n in melody_notes if m_start <= n.start < m_end]
            
            if measure_melody:
                avg_melody = np.mean([n.pitch for n in measure_melody])
                voicing = -12 if avg_melody > 72 else (12 if avg_melody < 54 else -7)
                num_voices = min(len(chord), int(2 + act["density"] * 2))
                
                if harmony_layer:
                    for j, interval in enumerate(chord[:num_voices]):
                        pitch = int(np.clip(key_root + interval + voicing, 36, 84))
                        arp_delay = j * 0.04
                        harm_vel = int(35 + 40 * act["intensity"]) + random.randint(-5, 5)
                        harmony_layer.notes.append(pretty_midi.Note(
                            velocity=harm_vel, pitch=pitch,
                            start=m_start + arp_delay, end=m_end
                        ))
                
                if strings_layer and measure % 2 == 0:
                    pitch = int(np.clip(key_root + chord[0] + voicing, 36, 84))
                    strings_layer.notes.append(pretty_midi.Note(
                        velocity=int(25 + 20 * act["intensity"]),
                        pitch=pitch, start=m_start, end=m_end + measure_dur * 0.3
                    ))
                
                if bass:
                    bass_root = int(np.clip(key_root + chord[0] - 24, 28, 52))
                    bass_vel = int(55 + 35 * act["intensity"])
                    bass.notes.append(pretty_midi.Note(
                        velocity=bass_vel, pitch=bass_root,
                        start=m_start, end=m_start + beat_dur * 3.5
                    ))
        
        if sparkle:
            for i, note in enumerate(melody_notes):
                progress = i / max(1, total_notes - 1)
                if 0.5 < progress < 0.75 and random.random() < 0.15:
                    sparkle_pitch = int(np.clip(note.pitch + 24, 84, 108))
                    sparkle.notes.append(pretty_midi.Note(
                        velocity=random.randint(35, 55), pitch=sparkle_pitch,
                        start=note.start, end=note.start + 0.4
                    ))
    
    # Clean layers
    if harmony_layer:
        harmony_layer.notes = clean_overlaps(harmony_layer.notes)
    if bass:
        bass.notes = clean_overlaps(bass.notes)
    
    # Assemble output
    output = pretty_midi.PrettyMIDI(initial_tempo=tempo)
    melody_raw.notes = melody_notes
    
    if bass:
        output.instruments.append(bass)
    if strings_layer:
        output.instruments.append(strings_layer)
    if harmony_layer:
        output.instruments.append(harmony_layer)
    output.instruments.append(melody_raw)
    if sparkle and len(sparkle.notes) > 5:
        output.instruments.append(sparkle)
    
    output.write(midi_out)
    
    print(f"\n✅ Complete! Duration: {output.get_end_time():.1f}s")
    print(f"📁 Saved to: {midi_out}")
    
    return midi_out, output


# ============================================================
# 🎨 INTERACTIVE UI
# ============================================================

def create_ui():
    """Create interactive Jupyter UI with all controls"""
    
    # Style
    display(HTML("""
    <style>
    .widget-label { font-weight: bold; color: #2c3e50; }
    .widget-dropdown select { font-size: 14px; }
    .widget-button { font-size: 16px !important; }
    </style>
    <h1 style='text-align:center; color:#8b5cf6; font-size:36px;'>
        🎬 Cinematic Piano Composer
    </h1>
    <p style='text-align:center; color:#6b7280; font-size:16px;'>
        Create movie-quality piano music with AI-powered emotional intelligence
    </p>
    <hr style='border: 2px solid #8b5cf6; width: 80%; margin: 20px auto;'>
    """))
    
    # Widgets
    emotion = widgets.Dropdown(
        options=[
            ('😢 Melancholy - Deep sadness', 'melancholy'),
            ('🌅 Hopeful - Warm & uplifting', 'hopeful'),
            ('🥀 Bittersweet - Sweet sadness', 'bittersweet'),
            ('🏆 Triumphant - Victory & power', 'triumphant'),
            ('📸 Nostalgic - Dreamy memories', 'nostalgic'),
            ('💕 Romantic - Intimate & loving', 'romantic'),
            ('⚔️ Epic - Grand & majestic', 'epic')
        ],
        value='bittersweet',
        description='🎭 Emotion:',
        style={'description_width': 'initial'}
    )
    
    story_arc = widgets.Dropdown(
        options=[
            ('⚔️ Hero\'s Journey', 'hero_journey'),
            ('💕 Love Story', 'love_story'),
            ('✨ Loss & Hope', 'loss_and_hope'),
            ('🔍 Mystery', 'mystery'),
            ('🌅 Gentle Dawn', 'gentle_dawn')
        ],
        value='hero_journey',
        description='📖 Story Arc:',
        style={'description_width': 'initial'}
    )
    
    key_root = widgets.Dropdown(
        options=[
            ('C Major - Pure, simple', 60),
            ('D Major - Bright, joyful', 62),
            ('E Major - Heroic, bold', 64),
            ('F Major - Peaceful', 65),
            ('G Major - Happy', 67),
            ('A Major - Confident', 69),
            ('A Minor - Melancholic', 57),
            ('G Minor - Dramatic', 55),
            ('B♭ Minor - Mysterious', 58)
        ],
        value=60,
        description='🎼 Key:',
        style={'description_width': 'initial'}
    )
    
    tempo = widgets.IntSlider(
        value=68,
        min=45,
        max=120,
        step=1,
        description='⏱️ Tempo (BPM):',
        style={'description_width': 'initial'}
    )
    
    clean_mode = widgets.Checkbox(
        value=True,
        description='🧹 Clean Mode (prevents overlaps)',
        style={'description_width': 'initial'}
    )
    
    dynamic_range = widgets.Dropdown(
        options=[
            ('Low - Gentle & subtle', 'low'),
            ('Moderate - Balanced', 'moderate'),
            ('High - Dramatic', 'high')
        ],
        value='moderate',
        description='📊 Dynamics:',
        style={'description_width': 'initial'}
    )
    
    phrase_length = widgets.Dropdown(
        options=[
            ('Short - Intimate', 'short'),
            ('Medium - Balanced', 'medium'),
            ('Long - Epic sweeps', 'long')
        ],
        value='medium',
        description='🎵 Phrases:',
        style={'description_width': 'initial'}
    )
    
    # Layer checkboxes
    layer_melody = widgets.Checkbox(value=True, description='🎹 Melody')
    layer_harmony = widgets.Checkbox(value=True, description='🎼 Harmony')
    layer_bass = widgets.Checkbox(value=True, description='🎸 Bass')
    layer_strings = widgets.Checkbox(value=True, description='🎻 Strings')
    layer_sparkle = widgets.Checkbox(value=True, description='✨ Sparkle')
    
    layers_box = widgets.HBox([layer_melody, layer_harmony, layer_bass, layer_strings, layer_sparkle])
    
    input_file = widgets.Text(
        value='final_rich_piano.mid',
        description='📂 Input MIDI:',
        style={'description_width': 'initial'}
    )
    
    output_file = widgets.Text(
        value='cinematic_output.mid',
        description='💾 Output MIDI:',
        style={'description_width': 'initial'}
    )
    
    generate_btn = widgets.Button(
        description='🎬 Generate Music',
        button_style='success',
        layout=widgets.Layout(width='300px', height='50px')
    )
    
    output_area = widgets.Output()
    
    def on_generate(b):
        with output_area:
            clear_output()
            print("🎵 Composing your cinematic masterpiece...\n")
            
            enable_layers = {
                "melody": layer_melody.value,
                "harmony": layer_harmony.value,
                "bass": layer_bass.value,
                "strings": layer_strings.value,
                "sparkle": layer_sparkle.value
            }
            
            try:
                midi_file, midi_obj = create_cinematic_piano(
                    midi_in=input_file.value,
                    midi_out=output_file.value,
                    emotion=emotion.value,
                    key_root=key_root.value,
                    tempo=tempo.value,
                    story_arc=story_arc.value,
                    clean_mode=clean_mode.value,
                    dynamic_range=dynamic_range.value,
                    phrase_length=phrase_length.value,
                    enable_layers=enable_layers
                )
                
                print("\n" + "="*60)
                print("🎉 SUCCESS! Your music is ready!")
                print("="*60)
                print(f"📁 File: {midi_file}")
                print(f"⏱️ Duration: {midi_obj.get_end_time():.1f} seconds")
                print(f"🎹 Instruments: {len(midi_obj.instruments)}")
                
                # Show enabled layers
                enabled = [k for k, v in enable_layers.items() if v]
                print(f"🎼 Layers: {', '.join(enabled)}")
                
            except Exception as e:
                print(f"❌ Error: {str(e)}")
                print("\n💡 Tip: Make sure your input MIDI file exists!")
    
    generate_btn.on_click(on_generate)
    
    # Layout
    display(widgets.VBox([
        widgets.HTML("<h3 style='color:#8b5cf6;'>🎭 Musical Settings</h3>"),
        emotion,
        story_arc,
        key_root,
        tempo,
        widgets.HTML("<h3 style='color:#8b5cf6; margin-top:20px;'>⚙️ Advanced Options</h3>"),
        clean_mode,
        dynamic_range,
        phrase_length,
        widgets.HTML("<h3 style='color:#8b5cf6; margin-top:20px;'>🎼 Instrument Layers</h3>"),
        layers_box,
        widgets.HTML("<h3 style='color:#8b5cf6; margin-top:20px;'>📁 Files</h3>"),
        input_file,
        output_file,
        widgets.HTML("<div style='text-align:center; margin:20px;'>"),
        generate_btn,
        widgets.HTML("</div>"),
        output_area
    ]))


# ============================================================
# 🚀 RUN THE UI
# ============================================================zq

if __name__ == "__main__":
    # If running in Jupyter, this will create the interactive UI
    try:
        get_ipython()  # This only works in Jupyter
        print("✅ Jupyter environment detected!")
        print("🎨 Creating interactive UI...\n")
        create_ui()
    except:
        # If running as script, use command line
        print("🎬 Running in script mode...")
        create_cinematic_piano(
            midi_in="final_rich_piano.mid",
            midi_out="cinematic_output.mid",
            emotion="romantic",
            story_arc="love_story",
            key_root=62,
            tempo=70,
            clean_mode=True,
            dynamic_range="moderate",
            phrase_length="medium"
        )

✅ Jupyter environment detected!
🎨 Creating interactive UI...



#### Story
##### ⚔️ Hero's Journey (4 acts: humble → challenge → triumph → peace)
##### 💕 Love Story (4 acts: meeting → growing → passion → forever)
##### ✨ Loss & Hope (4 acts: pain → darkness → darkest → light)
##### 🔍 Mystery (4 acts: question → investigate → reveal → understand)
##### 🌅 Gentle Dawn (4 acts: night → first light → sunrise → day)


#### key
##### C Major - Pure, simple
##### D Major - Bright, joyful
##### E Major - Heroic, bold
##### F Major - Peaceful
##### G Major - Happy
##### A Major - Confident
##### A Minor - Melancholic
##### G Minor - Dramatic
##### B♭ Minor - Mysterious

#### Tempo
##### Control speed of music
##### Speed:
##### 45-60 BPM = Very slow (ballads, emotional scenes)
##### 60-80 BPM = Slow/Medium (most cinematic music) ← You're here (68)
##### 80-100 BPM = Medium (walking pace)
##### 100-120 BPM = Fast (action scenes)

#### Clean Mode
##### Notes never overlap
##### Result: Clear, professional sound

#### Dynamics:
##### Controls musical breathing (how often there are pauses) like sentences in speech
##### Low Dynamics: Like whispering - everything is gentle
##### High Dynamics: Like a movie trailer - extreme contrasts


#### Phases
##### Short - Creates intimate, conversational feel with pauses every 8-16 notes
##### Medium - Natural flow with pauses every 16-32 notes 
##### Long - Grand, continuous phrases with pauses every 24-48 notes

